In [108]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Xi-lia Zheng et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['(K0 * ((n*n*n)/((1-n)*(1-n)))*(((1-n0)*(1-n0))/(n0*n0*n0)))/((n*n*n) / (c * ((1 - n)*(1 - n)) * (S*S)))']

model = ot.SymbolicFunction(['K0', 'n', 'n0', 'c', 'S'], formula)
distributions = [ot.Uniform(2.2e-4, 1e-3),  # initial saturated permeability coefficient before recharge during recharge [m/d]
                 ot.Uniform(0.5, 0.99),  # porosity of the sand media [-]
                 ot.Uniform(-3, 0.99),  # initial porosity of aquifer media [-]
                 ot.Uniform(3.52, 402),  # Korenzy constant based on solid volume
                 ot.Uniform(1, 10)]  # Specific surface area based on solid volume
distribution = ot.ComposedDistribution(distributions)

print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K0', 'n', 'n0', 'c', 'S']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K0', 'n', 'n0', 'c', 'S'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['K0', 'n', 'n0', 'c', 'S']
input_vars = ['K0', 'n', 'n0', 'c', 'S']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.00022, b = 0.001), Uniform(a = 0.5, b = 0.99), Uniform(a = -3, b = 0.99), Uniform(a = 3.52, b = 402), Uniform(a = 1, b = 10), IndependentCopula(dimension = 5))
[-0.00903959,-0.00870381,1,-0.0118258,-0.145612]
The least sensitive variable is:  S
The most sensitive variable is:  n0
The least sensitive variable is:  n
The most sensitive variable is:  n0
First-order indices after removing least sensitive variable:  [0.0151429,0.0121944,0.99627,0.0119104,0.0121944]
Total order indices after removing least sensitive variable:  [0.0128487,0,0.991669,0.401339,-3.0025e-16]
The least sensitive variable is:  S
Modified formula:  ['(K0 * ((n*n*n)/((1-n)*(1-n)))*(((1-n0)*(1-n0))/(n0*n0*n0)))/((n*n*n) / (c * ((1 - n)*(1 - n)) * (0.1*0.1)))']


Total Order Indices:  [0.00515316,-1.38359e-13,1.0087,0.0296913,0.43338]


In [109]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Schulz, Raphael (2020)

ot.RandomGenerator.SetSeed(0)

formula = ['(c0/(sigma_0*sigma_0))*((theta*theta*theta)/((1-theta)*(1-theta)))']

model = ot.SymbolicFunction(['c0', 'sigma_0', 'theta'], formula)
distributions = [ot.Uniform(3.52, 402), # Korenzy constant 
                 ot.Uniform(1, 10), # specific surface with respect to the unit volume of the solid matrix
                 ot.Uniform(-3, 0.99)] #  porosity
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['c0', 'sigma_0', 'theta']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['c0', 'sigma_0', 'theta'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())


print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['c0', 'sigma_0', 'theta']
input_vars = ['c0', 'sigma_0', 'theta']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()


ComposedDistribution(Uniform(a = 3.52, b = 402), Uniform(a = 1, b = 10), Uniform(a = -3, b = 0.99), IndependentCopula(dimension = 3))
[-0.0201689,-0.0203562,0.985809]
The least sensitive variable is:  sigma_0
The most sensitive variable is:  theta
The least sensitive variable is:  c0
The most sensitive variable is:  theta
First-order indices after removing least sensitive variable:  [0.0129261,0.0660091,0.896859]
Total order indices after removing least sensitive variable:  [0.0989939,0,0.904372]
Modified formula:  ['(c0/(0.1*0.1))*((theta*theta*theta)/((1-theta)*(1-theta)))']


Total Order Indices:  [0.0437378,0.645367,1.03409]


In [186]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Hubert J. Morel-Seytoux et al., 2016

ot.RandomGenerator.SetSeed(0)
formula = ['(((2*Kh)/(B+H))*(T/((1-(G/Daq))*T)))*(L*(B+H)*(hs-ha))']

model = ot.SymbolicFunction(['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha'], formula)
distributions = [ot.Uniform(0.9144, 457.2), #  horizontal hydraulic conductivity
                 ot.Uniform(10, 250), #  half width of the river
                 ot.Uniform(1.5, 7), # depth of water in river from water surface to streambed
                 ot.Uniform(1e-4, 1e-2), # clogging  layer  correction
                 ot.Uniform(90, 400), # Grid size
                 ot.Uniform(0.1, 5145), # Aquifer thickness
                 ot.Uniform(90, 400), #  length of the river reach within the river cell
                 ot.Uniform(1, 10), # head in river
                 ot.Uniform(130, 270)] #  head in aquifer
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())


print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha']
input_vars = ['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.9144, b = 457.2), Uniform(a = 10, b = 250), Uniform(a = 1.5, b = 7), Uniform(a = 0.0001, b = 0.01), Uniform(a = 90, b = 400), Uniform(a = 0.1, b = 5145), Uniform(a = 90, b = 400), Uniform(a = 1, b = 10), Uniform(a = 130, b = 270), IndependentCopula(dimension = 9))
[0.129918,-0.0164417,-0.0164417,-0.0164417,-0.034778,0.356889,-0.0591219,-0.0165791,0.0248323]
The least sensitive variable is:  L
The most sensitive variable is:  Daq
The least sensitive variable is:  T
The most sensitive variable is:  Daq
First-order indices after removing least sensitive variable:  [0.333992,0.0898557,0.0898557,0.0898557,0.0517256,0.253455,0.0898557,0.0905103,0.121157]
Total order indices after removing least sensitive variable:  [0.184801,-1.81019e-16,-1.81019e-16,-1.81019e-16,0.951163,0.912539,-1.81019e-16,0.000213985,0.0224992]
Modified formula:  ['(((2*Kh)/(B+H))*(T/((1-(G/Daq))*T)))*(0.1*(B+H)*(hs-ha))']


Total Order Indices:  [0.110153,7.5822e-16,7.5822e-16,5.68665e-16,0.551755,0.953538,0.225473,9.26974e-05,0.0312974]


In [11]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Rebecca C. Doble et al., 2012

ot.RandomGenerator.SetSeed(0)

formula = ['(Kc(hw + dc))/(dc*dgw*((Sy/tw)+((2*Taq)/(xw*xw))))']

model = ot.SymbolicFunction(['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw'], formula)
distributions = [ot.Uniform(0.017, 7.13), #  Saturated hydraulic conductivity of the surface soil or clogging layer [LT-1]
                 ot.Uniform(10, 250), #  height of the wave above the bankfull elevation
                 ot.Uniform(1.5, 7), # Thickness of the clogging layer
                 ot.Uniform(1e-4, 1e-2), # depth of groundwater = taken at the centre of the flood wave extent
                 ot.Uniform(90, 400), # Aquifer specific yield
                 ot.Uniform(0.1, 5145), # duration of the flood wave
                 ot.Uniform(90, 400), #  Aquifer transmissivity
                 ot.Uniform(130, 270)] # lateral extent of flooding
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw']
input_vars = ['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size, family="Arial, bold"))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size, family="Arial, bold"))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-1, max(y) + 0.1])
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='blue'))
# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='red'))
annotation = []

layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=24),
                   xaxis=dict(title='Input Variables', title_font=dict(size=20), tickfont=dict(size=20), automargin=True),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=20), tickfont=dict(size=20), automargin=True, tickformat='.2f', range=[-0.2, 1]),
                   annotations=annotation,
                   legend=dict(x=0, y=-0.2, xanchor='left', yanchor='bottom', font=dict(size=15)))

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.017, b = 7.13), Uniform(a = 10, b = 250), Uniform(a = 1.5, b = 7), Uniform(a = 0.0001, b = 0.01), Uniform(a = 90, b = 400), Uniform(a = 0.1, b = 5145), Uniform(a = 90, b = 400), Uniform(a = 130, b = 270), IndependentCopula(dimension = 8))
[0.036047,0.033823,0.0700048,0.437864,-0.0160319,0.0676353,0.00989235,0.00268777]
The least sensitive variable is:  Sy
The most sensitive variable is:  dgw
The least sensitive variable is:  xw
The most sensitive variable is:  dgw
First-order indices after removing least sensitive variable:  [0.0301772,-0.0313336,-0.0127174,0.896767,-0.019216,-0.0189167,0.0723795,-0.0317401]
Total order indices after removing least sensitive variable:  [0.140395,0.0167323,0.166773,0.970954,0,2.23466e-05,0.0149032,0.0266576]
The least sensitive variable is:  Sy
Modified formula:  ['(Kc(hw + dc))/(dc*dgw*((0.1/tw)+((2*Taq)/(xw*xw))))']


Total Order Indices:  [0.469555,0.433652,0.122725,0.89932,0.0360859,0.0523304,0.00366115,0.00173142]


In [13]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Pedretti et al. 2012

ot.RandomGenerator.SetSeed(0)

formula = ['I0 * (2.7182818285^(-(mp * lambda_z * va * t)))']

model = ot.SymbolicFunction(['I0', 'mp', 'lambda_z', 'va', 't'], formula)
distributions = [ot.Uniform(1e-5, 0.002), # Infiltration capacity [LT-1]
                 ot.Uniform(1, 100), # coefficient of proportionality [-]
                 ot.Uniform(0.044, 0.363), # Filtration coefficient [L-1]
                 ot.Uniform(1e-5, 1.1e-5), # Average particle attachment velocity to the soil matrix
                 ot.Uniform(1, 365)] # time
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['I0', 'mp', 'lambda_z', 'va', 't']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['I0', 'mp', 'lambda_z', 'va', 't'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['I0', 'mp', 'lambda_z', 'va', 't']
input_vars = ['I0', 'mp', 'lambda_z', 'va', 't']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='blue'))
# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='red'))
annotation = []

layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=24),
                   xaxis=dict(title='Input Variables', title_font=dict(size=20), tickfont=dict(size=20), automargin=True),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=20), tickfont=dict(size=20), automargin=True, tickformat='.2f', range=[-0.2, 1.3]),
                   annotations=annotation,
                   legend=dict(x=0, y=-0.2, xanchor='left', yanchor='bottom', font=dict(size=15)))

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1e-05, b = 0.002), Uniform(a = 1, b = 100), Uniform(a = 0.044, b = 0.363), Uniform(a = 1e-05, b = 1.1e-05), Uniform(a = 1, b = 365), IndependentCopula(dimension = 5))
[0.999156,0.136031,0.139958,0.13749,0.13731]
The least sensitive variable is:  mp
The most sensitive variable is:  I0
The least sensitive variable is:  va
The most sensitive variable is:  I0
First-order indices after removing least sensitive variable:  [1,-0.204101,-0.204103,-0.204102,-0.204099]
Total order indices after removing least sensitive variable:  [1.20408,-4.59496e-16,1.90881e-09,1.17236e-11,2.81059e-09]
The least sensitive variable is:  mp
Modified formula:  ['I0 * (2.7182818285^(-(0.1 * lambda_z * va * t)))']


Total Order Indices:  [0.870537,0.000933148,0.00035092,2.78515e-06,0.000641881]


In [113]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Jung-Woo Kim et al., 2009

ot.RandomGenerator.SetSeed(0)

formula = ['((T*(4*3.141592654*delta_h))/2.3)/(delta_h/(Lt*A))'] 

model = ot.SymbolicFunction(['T', 'delta_h', 'Lt', 'A'], formula)
distributions = [ot.Uniform(1860, 2240), # Transmissivity
                 ot.Uniform(0.001, 4.1), # hydraulic head loss [mm]
                 ot.Uniform(130, 270),# length of the glass bead layer [mm]
                 ot.Uniform(90, 400)] # cross-sectional area of the glass bead layer [mm2]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['T', 'delta_h', 'Lt', 'A']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['T', 'delta_h', 'Lt', 'A'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['T', 'delta_h', 'Lt', 'A']
input_vars = ['T', 'delta_h', 'Lt', 'A']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1860, b = 2240), Uniform(a = 0.001, b = 4.1), Uniform(a = 130, b = 270), Uniform(a = 90, b = 400), IndependentCopula(dimension = 4))
[0.0648774,0.0618126,0.248268,0.585619]
The least sensitive variable is:  delta_h
The most sensitive variable is:  A
The least sensitive variable is:  delta_h
The most sensitive variable is:  A
First-order indices after removing least sensitive variable:  [-0.00576555,-0.0209149,0.373041,0.652473]
Total order indices after removing least sensitive variable:  [0.0179644,-3.36285e-16,0.314736,0.70832]
The least sensitive variable is:  delta_h
Modified formula:  ['((T*(4*3.141592654*0.1))/2.3)/(0.1/(Lt*A))']


Total Order Indices:  [0.0164657,-2.72868e-16,0.283935,0.645615]


In [114]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Jena-Olaf Delfs 2011

ot.RandomGenerator.SetSeed(0)

formula = ['delta_1*(2.71828128*(delta_2*(h-hg)))'] 

model = ot.SymbolicFunction(['delta_1', 'delta_2', 'h', 'hg'], formula)
distributions = [ot.Uniform(1e-1, 1), # Λ1 and Λ2 are parameters and implemented into OGS code.
                 ot.Uniform(0.005, 0.1), 
                 ot.Uniform(1, 5), # h = H + b the water height in the river
                 ot.Uniform(1, 10)] # water table height
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['delta_1', 'delta_2', 'h', 'hg']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['delta_1', 'delta_2', 'h', 'hg'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['delta_1', 'delta_2', 'h', 'hg']
input_vars = ['delta_1', 'delta_2', 'h', 'hg']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 1), Uniform(a = 0.005, b = 0.1), Uniform(a = 1, b = 5), Uniform(a = 1, b = 10), IndependentCopula(dimension = 4))
[0.091938,0.252023,0.254644,0.362292]
The least sensitive variable is:  delta_1
The most sensitive variable is:  hg
The least sensitive variable is:  delta_1
The most sensitive variable is:  hg
First-order indices after removing least sensitive variable:  [0.150803,0.410453,0.230992,0.586284]
Total order indices after removing least sensitive variable:  [1.96796e-16,0.388793,0.149777,0.524644]
The least sensitive variable is:  delta_1
Modified formula:  ['0.1*(2.71828128*(delta_2*(h-hg)))']


Total Order Indices:  [0.182039,0.376505,0.186083,0.666798]


In [115]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
ot.Log.Show(ot.Log.NONE)

# Xuezhen Zhang et al., 2020

ot.RandomGenerator.SetSeed(0)

formula = ['(Et0 - Et)/Et0'] 

model = ot.SymbolicFunction(['Et0', 'Et'], formula)
distributions = [ot.Uniform(0.143, 1.194), # initial water absorbing capacity
                 ot.Uniform(6.056, 6.282)] # water absorbing capacity at time t
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Et0', 'Et']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.0001')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Et0', 'Et'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Et0', 'Et']
input_vars = ['Et0', 'Et']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.143, b = 1.194), Uniform(a = 6.056, b = 6.282), IndependentCopula(dimension = 2))
[0.999656,-0.0355463]
The least sensitive variable is:  Et
The most sensitive variable is:  Et0
The least sensitive variable is:  Et
The most sensitive variable is:  Et0


TypeError: InvalidArgumentException : Null output sample variance

In [116]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

#Xinqiang Du et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*l)/(3.141592654*(r^2)*d_h)'] 

model = ot.SymbolicFunction(['Q', 'l', 'r', 'd_h'], formula)
distributions = [ot.Uniform(0.004992, 840), # flow rate [m3d-1]
                 ot.Uniform(1, 10), # distance between any two piezometric tubes along the column [m]
                 ot.Uniform(1, 5), # Inner diameter of the column [m]
                 ot.Uniform(0.001, 4.1)] #  hydraulic head difference at a difference l [m]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'l', 'r', 'd_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'l', 'r', 'd_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Q', 'l', 'r', 'd_h']
input_vars = ['Q', 'l', 'r', 'd_h']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.0591762,0.0333257,0.0419325,0.368628]
The least sensitive variable is:  l
The most sensitive variable is:  d_h
The least sensitive variable is:  Q
The most sensitive variable is:  d_h
First-order indices after removing least sensitive variable:  [0.0123108,-0.00530018,0.0374737,0.298991]
Total order indices after removing least sensitive variable:  [0.03271,0,0.213422,0.761323]
The least sensitive variable is:  l
Modified formula:  ['(Q*0.1)/(3.141592654*(r^2)*d_h)']


Total Order Indices:  [0.00444871,0.171589,0.451128,0.872661]


In [117]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Kristin J. Rehg et al., 2004

ot.RandomGenerator.SetSeed(0)

formula = ['d/((m*theta)+d)'] 

model = ot.SymbolicFunction(['d', 'm', 'theta'], formula)
distributions = [ot.Uniform(1.5, 10.87), # effective stream depth (total volume of stream water per unit bed area)
                 ot.Uniform(0, 0.18), # average depth of solute penetration into the bed
                 ot.Uniform(-3, 0.99)] # bulk porosity
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['d', 'm', 'theta']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['d', 'm', 'theta'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['d', 'm', 'theta']
input_vars = ['d', 'm', 'theta']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1.5, b = 10.87), Uniform(a = 0, b = 0.18), Uniform(a = -3, b = 0.99), IndependentCopula(dimension = 3))
[-0.128585,-0.0601137,0.3872]
The least sensitive variable is:  d
The most sensitive variable is:  theta
The least sensitive variable is:  d
The most sensitive variable is:  theta
First-order indices after removing least sensitive variable:  [-0.0493395,-0.00470377,0.0421137]
Total order indices after removing least sensitive variable:  [2.05753e-16,0.946556,1.08424]
The least sensitive variable is:  d
Modified formula:  ['0.1/((m*theta)+0.1)']


Total Order Indices:  [0.373459,0.627742,0.7015]


In [118]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# E. Hoehn et al., 2006

ot.RandomGenerator.SetSeed(0)

formula = ['(x*ne)/tao_w'] 

model = ot.SymbolicFunction(['x', 'ne', 'tao_w'], formula)
distributions = [ot.Uniform(1, 10), # path length from the infiltration point in the river to a nearby monitoring well
                 ot.Uniform(-3, 0.99), # flow-effective porosity of the riverbed and aquifer materials
                 ot.Uniform(1, 365)] # mean residence time of freshly infiltrated water
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['x', 'ne', 'tao_w']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['x', 'ne', 'tao_w'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['x', 'ne', 'tao_w']
input_vars = ['x', 'ne', 'tao_w']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1, b = 10), Uniform(a = -3, b = 0.99), Uniform(a = 1, b = 365), IndependentCopula(dimension = 3))
[-0.0128778,0.0191238,-0.391791]
The least sensitive variable is:  tao_w
The most sensitive variable is:  ne
The least sensitive variable is:  ne
The most sensitive variable is:  tao_w
First-order indices after removing least sensitive variable:  [-0.0155298,0.703508,-0.0118076]
Total order indices after removing least sensitive variable:  [0.336184,0.802738,5.93729e-16]
The least sensitive variable is:  tao_w
Modified formula:  ['(x*ne)/0.1']


Total Order Indices:  [0.102935,0.0445288,1.00068]


In [119]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

#Konstantinos N. Moutsopoulos 2013

ot.RandomGenerator.SetSeed(0)

formula = ['(K*b)/k'] 

model = ot.SymbolicFunction(['K', 'b', 'k'], formula)
distributions = [ot.Uniform(1e-12, 1e0), # hydraulic conductivity of the aquifer 
                 ot.Uniform(1, 10), # length of the base of semipervious layer of rectangular shape
                 ot.Uniform(0.1, 50)] # the hydraulic conductivity of the clogging layer (m/d).
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K', 'b', 'k']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K', 'b', 'k'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['K', 'b', 'k']
input_vars = ['K', 'b', 'k']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1e-12, b = 1), Uniform(a = 1, b = 10), Uniform(a = 0.1, b = 50), IndependentCopula(dimension = 3))
[-0.0438464,-0.0496101,0.564772]
The least sensitive variable is:  b
The most sensitive variable is:  k
The least sensitive variable is:  b
The most sensitive variable is:  k
First-order indices after removing least sensitive variable:  [-0.0279695,-0.0697694,0.891142]
Total order indices after removing least sensitive variable:  [0.634342,-3.39972e-16,1.03177]
The least sensitive variable is:  b
Modified formula:  ['(K*0.1)/k']


Total Order Indices:  [0.254368,0.153699,1.02834]


In [120]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Sebastian Rudnick et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['1/(G*(1/(1+(2.718281828^(-k*G*norm))*((G/k0)-1)))*clogg_max)'] 

model = ot.SymbolicFunction(['G', 'k', 'norm', 'k0', 'clogg_max'], formula) # G = 1, k = 12, and k0 = 0.02
distributions = [ot.Uniform(0.9, 1), # soil heat-flux density (MJ/[m2*d]),
                 ot.Uniform(1, 12), # hydraulic gradients
                 ot.Uniform(-0.26, -0.25), # vertical hydraulic gradient of the cell normalized by the steepest negative gradient of −0.25 m/m
                 ot.Uniform(0.01, 0.02), # hydraulic gradient at 0
                 ot.Uniform(1, 100)] # cloggmax is the maximum assumed clogging factor of 100 (–)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['G', 'k', 'norm', 'k0', 'clogg_max']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['G', 'k', 'norm', 'k0', 'clogg_max'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['G', 'k', 'norm', 'k0', 'clogg_max']
input_vars = ['G', 'k', 'norm', 'k0', 'clogg_max']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.9, b = 1), Uniform(a = 1, b = 12), Uniform(a = -0.26, b = -0.25), Uniform(a = 0.01, b = 0.02), Uniform(a = 1, b = 100), IndependentCopula(dimension = 5))
[-0.00828519,0.0505858,-0.00702523,0.0028972,0.57688]
The least sensitive variable is:  G
The most sensitive variable is:  clogg_max
The least sensitive variable is:  norm
The most sensitive variable is:  clogg_max
First-order indices after removing least sensitive variable:  [-0.0275833,-0.0356542,-0.0274681,-0.00769843,0.981265]
Total order indices after removing least sensitive variable:  [0,0.0071551,1.78227e-06,0.017287,1.01591]
The least sensitive variable is:  G
Modified formula:  ['1/(0.1*(1/(1+(2.718281828^(-k*0.1*norm))*((0.1/k0)-1)))*clogg_max)']


Total Order Indices:  [0.00361901,0.402405,0.000268988,0.00992127,0.947734]


In [14]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Vassilios Kaleris 1998

ot.RandomGenerator.SetSeed(0)

formula = ['((a-psi_ij)*A)/(RL_ij)'] 

model = ot.SymbolicFunction(['a', 'psi_ij', 'A', 'RL_ij'], formula)
distributions = [ot.Uniform(1.5, 7), # water depth in the stream [m]
                 ot.Uniform(1, 2), # pressure head below the clogging layer [m]
                 ot.Uniform(90, 400), # stream bed area corresponding to the node (i j) = [m2]
                 ot.Uniform(1e-12, 1e0)] # hydraulic resistance of the clogging layer at the node (i, j) = [s-1]

distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['a', 'psi_ij', 'A', 'RL_ij']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['a', 'psi_ij', 'A', 'RL_ij'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['a', 'psi_ij', 'A', 'RL_ij']
input_vars = ['a', 'psi_ij', 'A', 'RL_ij']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='blue'))
# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='red'))
annotation = []

layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=24),
                   xaxis=dict(title='Input Variables', title_font=dict(size=20), tickfont=dict(size=20), automargin=True),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=20), tickfont=dict(size=20), automargin=True, tickformat='.2f', range=[-0.2, 1]),
                   annotations=annotation,
                   legend=dict(x=0, y=-0.2, xanchor='left', yanchor='bottom', font=dict(size=15)))

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1.5, b = 7), Uniform(a = 1, b = 2), Uniform(a = 90, b = 400), Uniform(a = 1e-12, b = 1), IndependentCopula(dimension = 4))
[0.108492,0.117599,0.113425,0.71928]
The least sensitive variable is:  a
The most sensitive variable is:  RL_ij
The least sensitive variable is:  psi_ij
The most sensitive variable is:  RL_ij
First-order indices after removing least sensitive variable:  [-0.0109299,-0.0040626,0.00217798,0.967399]
Total order indices after removing least sensitive variable:  [-9.28879e-16,0.0862424,0.0681294,1.02027]
The least sensitive variable is:  a
Modified formula:  ['((0.1-psi_ij)*A)/(RL_ij)']


Total Order Indices:  [0.0105749,0.00621088,0.0177414,0.756216]


In [15]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

#M. Dimkic et al., 2011

ot.RandomGenerator.SetSeed(0)

formula = ['(K_l/d)*Delta_S*2*3.141592654*r*L'] 

model = ot.SymbolicFunction(['K_l', 'd', 'Delta_S', 'r', 'L'], formula)
distributions = [ot.Uniform(0.1, 50), #  Representative Values of the Coefficient of Local Hydraulic Losses at Well Laterals, KL /d conductivity of the well-lateral colmated layer [L T−1]
                 ot.Uniform(0.1, 0.11), # thickness of the colmated layer [L]
                 ot.Uniform(0.001, 4.1), # piezometric head drop at the colmated layer, the local drawdown
                 ot.Uniform(1, 5), # radius of the well lateral [L]
                 ot.Uniform(1, 10)] # length of the well lateral [L]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K_l', 'd', 'Delta_S', 'r', 'L']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K_l', 'd', 'Delta_S', 'r', 'L'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['K_l', 'd', 'Delta_S', 'r', 'L']
input_vars = ['K_l', 'd', 'Delta_S', 'r', 'L']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()
# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='blue'))
# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    textfont=dict(size=16),
                    texttemplate='%{text:.2f}',
                    textangle=0,
                    marker=dict(color='red'))
annotation = []

layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=24),
                   xaxis=dict(title='Input Variables', title_font=dict(size=20), tickfont=dict(size=20), automargin=True),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=20), tickfont=dict(size=20), automargin=True, tickformat='.2f', range=[-0.2, 1]),
                   annotations=annotation,
                   legend=dict(x=0, y=-0.2, xanchor='left', yanchor='bottom', font=dict(size=15)))

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 50), Uniform(a = 0.1, b = 0.11), Uniform(a = 0.001, b = 4.1), Uniform(a = 1, b = 5), Uniform(a = 1, b = 10), IndependentCopula(dimension = 5))
[0.329926,0.0994834,0.19751,0.133271,0.209995]
The least sensitive variable is:  d
The most sensitive variable is:  K_l
The least sensitive variable is:  d
The most sensitive variable is:  L
First-order indices after removing least sensitive variable:  [0.151239,-0.0734726,0.220595,0.0603313,0.0538236]
Total order indices after removing least sensitive variable:  [0.660711,1.12872e-16,0.617057,0.312157,0.315331]
The least sensitive variable is:  d
Modified formula:  ['(K_l/0.1)*Delta_S*2*3.141592654*r*L']


Total Order Indices:  [0.203095,0.00115416,0.244717,0.188708,0.309647]


In [123]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Hong Hanh Nguyen et al., 2021

ot.RandomGenerator.SetSeed(0)

formula = ['h*(k_c/B)*(W_p*L*p_leak)'] 

model = ot.SymbolicFunction(['h', 'k_c', 'B', 'W_p', 'L', 'p_leak'], formula)
distributions = [ot.Uniform(0, 0.1), # wastewater level in the pipe (L)
                 ot.Uniform(0.1, 50), # hydraulic conductivity of colmation layer (L T−1)
                 ot.Uniform(0.1, 0.11), # thickness of the colmation layer (L)
                 ot.Uniform(99.9, 99.999), # wetted perimeter generated with the water level in the sewer (L)
                 ot.Uniform(1, 10), # length of the sewer segment of interest (L)
                 ot.Uniform(0.005, 0.1)] # percentage of leaking surfaces from the total area under the wetted perimeter.
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['h', 'k_c', 'B', 'W_p', 'L', 'p_leak']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['h', 'k_c', 'B', 'W_p', 'L', 'p_leak'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['h', 'k_c', 'B', 'W_p', 'L', 'p_leak']
input_vars = ['h', 'k_c', 'B', 'W_p', 'L', 'p_leak']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0, b = 0.1), Uniform(a = 0.1, b = 50), Uniform(a = 0.1, b = 0.11), Uniform(a = 99.9, b = 99.999), Uniform(a = 1, b = 10), Uniform(a = 0.005, b = 0.1), IndependentCopula(dimension = 6))
[0.268363,0.182783,-0.00227884,-0.00993451,0.0387578,0.116153]
The least sensitive variable is:  W_p
The most sensitive variable is:  h
The least sensitive variable is:  W_p
The most sensitive variable is:  h
First-order indices after removing least sensitive variable:  [0.503517,0.454298,0.345984,0.329781,0.375093,0.439567]
Total order indices after removing least sensitive variable:  [0.321005,0.324899,0.0010441,2.97042e-16,0.19395,0.29791]
The least sensitive variable is:  W_p
Modified formula:  ['h*(k_c/B)*(0.1*L*p_leak)']


Total Order Indices:  [0.386614,0.359333,0.00152919,8.62527e-08,0.239546,0.228167]


In [124]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Mohamed Amine Boukhemacha et al., 2015

ot.RandomGenerator.SetSeed(0)

formula = ['K*(delta_H/B)*(W_p*L*p_leaks)'] 

model = ot.SymbolicFunction(['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks'], formula)
distributions = [ot.Uniform(0.1, 50), # hydraulic conductivity of the clogging layer [m/day] = 0.15 – 33.09 (literature) = 7.34 – 29.37 (NEIMO calibration)
                 ot.Uniform(0, 0.1), # hydraulic head difference between the water level in the sewer and the hydraulic head of the surrounding groundwater (taken to be null) [m]
                 ot.Uniform(0.1, 0.11), # thickness of the clogging layer [m] = 0.1
                 ot.Uniform(99.9, 99.999), # wetted parameter
                 ot.Uniform(1, 10), # length
                 ot.Uniform(0.005, 0.1)] # percentage of the leaky area from the total area generated by the wetted perimeter
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks']
input_vars = ['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 50), Uniform(a = 0, b = 0.1), Uniform(a = 0.1, b = 0.11), Uniform(a = 99.9, b = 99.999), Uniform(a = 1, b = 10), Uniform(a = 0.005, b = 0.1), IndependentCopula(dimension = 6))
[0.266466,0.184559,-0.00222142,-0.00990753,0.0392449,0.116777]
The least sensitive variable is:  W_p
The most sensitive variable is:  K
The least sensitive variable is:  W_p
The most sensitive variable is:  K
First-order indices after removing least sensitive variable:  [0.502451,0.455119,0.345814,0.329618,0.374831,0.439457]
Total order indices after removing least sensitive variable:  [0.318892,0.327085,0.00104319,-2.97259e-16,0.194025,0.297932]
The least sensitive variable is:  W_p
Modified formula:  ['K*(delta_H/B)*(0.1*L*p_leaks)']


Total Order Indices:  [0.384269,0.361532,0.00152844,8.6135e-08,0.239273,0.22816]


In [125]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Stephane Descloux et al., 2010

ot.RandomGenerator.SetSeed(0)

formula = ['(((Mt-sum_Mw)*p)/Vt)*100'] 

model = ot.SymbolicFunction(['Mt', 'sum_Mw', 'p', 'Vt'], formula)
distributions = [ot.Uniform(1, 10), # Total mass of sample core [kg]
                 ot.Uniform(1, 5), # Mass of wet sediments
                 ot.Uniform(999.9, 1000), # Water density [kg/m3] = 1000
                 ot.Uniform(1, 10)] #  Total volume of sample core
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Mt', 'sum_Mw', 'p', 'Vt']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Mt', 'sum_Mw', 'p', 'Vt'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Mt', 'sum_Mw', 'p', 'Vt']
input_vars = ['Mt', 'sum_Mw', 'p', 'Vt']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 999.9, b = 1000), Uniform(a = 1, b = 10), IndependentCopula(dimension = 4))
[0.534418,0.0945905,0.0528694,0.3402]
The least sensitive variable is:  p
The most sensitive variable is:  Mt
The least sensitive variable is:  p
The most sensitive variable is:  Vt
First-order indices after removing least sensitive variable:  [0.499166,0.125385,0.10839,0.33692]
Total order indices after removing least sensitive variable:  [0.427536,0.118886,3.98241e-16,0.457101]
The least sensitive variable is:  p
Modified formula:  ['(((Mt-sum_Mw)*0.1)/Vt)*100']


Total Order Indices:  [0.487096,0.148534,1.49639e-09,0.498562]


In [126]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# K. Pholkern et al., 2015

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*L)/(A*delta_h)'] 

model = ot.SymbolicFunction(['Q', 'L', 'A', 'delta_h'], formula)
distributions = [ot.Uniform(0.004992, 840), # Outflow / Infiltration discharge [m3/s]
                 ot.Uniform(0.003, 0.006), #Thickness of clogging layer
                 ot.Uniform(5.65, 237.56), # Area of flow [m2]
                 ot.Uniform(0.001, 4.1)] #  Head difference [m]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'L', 'A', 'delta_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'L', 'A', 'delta_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Q', 'L', 'A', 'delta_h']
input_vars = ['Q', 'L', 'A', 'delta_h']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 0.003, b = 0.006), Uniform(a = 5.65, b = 237.56), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.020594,0.0107956,0.0329896,0.267529]
The least sensitive variable is:  L
The most sensitive variable is:  delta_h
The least sensitive variable is:  Q
The most sensitive variable is:  delta_h
First-order indices after removing least sensitive variable:  [0.00593693,-0.0129998,0.0768694,0.284839]
Total order indices after removing least sensitive variable:  [0.0508459,4.32186e-16,0.141652,0.802564]
The least sensitive variable is:  L
Modified formula:  ['(Q*0.1)/(A*delta_h)']


Total Order Indices:  [0.00347053,0.0227105,0.514633,0.842007]


In [188]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

#czuba
ot.RandomGenerator.SetSeed(0)
formula = ['((As/(R*tc))^(1/18))*(((Qb*g)/(a*kn*Bb))^(1/3))*(S^(7/18))'] 

model = ot.SymbolicFunction(['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S'], formula)
distributions = [ot.Uniform(0.5, 0.6), # constant
                 ot.Uniform(0.6, 1.65), #Submurged specific gravity of sediment
                 ot.Uniform(0.03, 0.6), # dimensionless critical bed shear stress (or Shields parameter) for initiation of motion
                 ot.Uniform(0.004992, 840), # denotes that these  values are at the specific flow condition when Dx is mobilized.
                 ot.Uniform(9.8, 9.81, ), # gravity
                 ot.Uniform(3, 3.3), # the coefficient of the Manning–Strickler form of the logarithmic law for estimating the velocity distribution in the vertical direction
                 ot.Uniform(1, 3.3), # conversion factor between SI and English units
                 ot.Uniform(3, 3.3), # channel width
                 ot.Uniform(3.5, 5)] #  Channel slope
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.FAST(model, distribution, 101)
print(sensitivityAnalysis.getFirstOrderIndices())

y = np.array(sensitivityAnalysis.getFirstOrderIndices())
mylab = ['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S']
# Set options for the pie chart size
pie_width = 900
pie_height = 900

# Create Plotly pie chart
fig = go.Figure(data=[go.Pie(labels=mylab,
                             values=y,
                             text=y,
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             textfont=dict(size=27), # set font size for all text
                             showlegend=False)]) # remove the legend

fig.update_layout(
    autosize=False,  # disable auto-sizing
    width=pie_width,  # set width
    height=pie_height,  # set height
)
fig.show()


ComposedDistribution(Uniform(a = 0.5, b = 0.6), Uniform(a = 0.6, b = 1.65), Uniform(a = 0.03, b = 0.6), Uniform(a = 0.004992, b = 840), Uniform(a = 9.8, b = 9.81), Uniform(a = 3, b = 3.3), Uniform(a = 1, b = 3.3), Uniform(a = 3, b = 3.3), Uniform(a = 3.5, b = 5), IndependentCopula(dimension = 9))
[0.000427843,0.0242003,0.031395,0.76893,0.000536143,0.001324,0.223099,0.00145092,0.057238]


In [127]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Jonathan A. Czuba et al., 2021

ot.RandomGenerator.SetSeed(0)

formula = ['((As/(R*tc))^(1/18))*(((Qb*g)/(a*kn*Bb))^(1/3))*(S^(7/18))'] 

model = ot.SymbolicFunction(['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S'], formula)
distributions = [ot.Uniform(0.5, 0.6), # constant
                 ot.Uniform(0.6, 1.65), #Submurged specific gravity of sediment
                 ot.Uniform(0.03, 0.6), # dimensionless critical bed shear stress (or Shields parameter) for initiation of motion
                 ot.Uniform(0.004992, 840), # denotes that these  values are at the specific flow condition when Dx is mobilized.
                 ot.Uniform(9.8, 9.81, ), # gravity
                 ot.Uniform(3, 3.3), # the coefficient of the Manning–Strickler form of the logarithmic law for estimating the velocity distribution in the vertical direction
                 ot.Uniform(1, 3.3), # conversion factor between SI and English units
                 ot.Uniform(3, 3.3), # channel width
                 ot.Uniform(3.5, 5)] #  Channel slope
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S']
input_vars = ['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.5, b = 0.6), Uniform(a = 0.6, b = 1.65), Uniform(a = 0.03, b = 0.6), Uniform(a = 0.004992, b = 840), Uniform(a = 9.8, b = 9.81), Uniform(a = 3, b = 3.3), Uniform(a = 1, b = 3.3), Uniform(a = 3, b = 3.3), Uniform(a = 3.5, b = 5), IndependentCopula(dimension = 9))
[0.105732,0.106207,0.154237,0.712589,0.105891,0.103065,0.255798,0.112279,0.119917]
The least sensitive variable is:  a
The most sensitive variable is:  Qb
The least sensitive variable is:  g
The most sensitive variable is:  Qb
First-order indices after removing least sensitive variable:  [-0.0417975,-0.033906,-0.0107761,0.783356,-0.0420187,-0.042045,0.0627181,-0.0395349,-0.0348003]
Total order indices after removing least sensitive variable:  [0.00012134,0.00352525,0.0217879,0.807672,1.06212e-07,-1.61415e-16,0.162021,0.000824283,0.0166113]
The least sensitive variable is:  a
Modified formula:  ['((As/(R*tc))^(1/18))*(((Qb*g)/(0.1*kn*Bb))^(1/3))*(S^(7/18))']


Total Order Indices:  [0.00010608,0.00238757,0.0173114,0.657579,1.28967e-07,0.000975316,0.135997,0.000894224,0.0171171]


In [128]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# STEPHANIE RINCK-PFEIFFER et al., 1999

ot.RandomGenerator.SetSeed(0)

formula = ['Q*(delta_x/(3.141592654*(r^2)))*delta_h'] 

model = ot.SymbolicFunction(['Q', 'delta_x', 'r', 'delta_h'], formula)
distributions = [ot.Uniform(0.004992, 840), #m3/d, Flowrate
                 ot.Uniform(1, 10), #m length of the column
                 ot.Uniform(1, 5), #m radius
                 ot.Uniform(0.001, 4.1)] #m hydraulic head difference
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'delta_x', 'r', 'delta_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'delta_x', 'r', 'delta_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Q', 'delta_x', 'r', 'delta_h']
input_vars = ['Q', 'delta_x', 'r', 'delta_h']
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.0668982,0.0425869,0.579263,0.00201209]
The least sensitive variable is:  delta_h
The most sensitive variable is:  r
The least sensitive variable is:  Q
The most sensitive variable is:  r
First-order indices after removing least sensitive variable:  [0.0137541,0.108376,0.394785,-0.0370068]
Total order indices after removing least sensitive variable:  [0.154702,0.226437,0.83255,-3.84768e-16]
The least sensitive variable is:  delta_h
Modified formula:  ['Q*(delta_x/(3.141592654*(r^2)))*0.1']


Total Order Indices:  [0.19032,0.405161,0.78121,0.365927]


In [129]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Ali rifat cabalar 2016

ot.RandomGenerator.SetSeed(0)

formula = ['6*(10^(-4))*(g/v)*(1+10*(n-0.026))*(d_10^2)'] 

model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)
distributions = [ot.Uniform(9.8, 9.81),
                 ot.Uniform(1.16, 4.39),
                 ot.Uniform(0.34, 0.47),
                 ot.Uniform(0.09, 2.20)]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['g', 'v', 'n', 'd_10']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['g', 'v', 'n', 'd_10']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 9.8, b = 9.81), Uniform(a = 1.16, b = 4.39), Uniform(a = 0.34, b = 0.47), Uniform(a = 0.09, b = 2.2), IndependentCopula(dimension = 4))
[-0.0819419,0.123099,-0.0883531,0.650553]
The least sensitive variable is:  n
The most sensitive variable is:  d_10
The least sensitive variable is:  g
The most sensitive variable is:  d_10
First-order indices after removing least sensitive variable:  [0.0682256,0.357191,0.0683116,0.732095]
Total order indices after removing least sensitive variable:  [2.01463e-07,0.33072,3.86112e-16,0.842268]
The least sensitive variable is:  n
Modified formula:  ['6*(10^(-4))*(g/v)*(1+10*(0.1-0.026))*(d_10^2)']


Total Order Indices:  [1.92516e-07,0.260732,0.0105105,0.864061]


In [130]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Ali Firat Cabalar 2016

ot.RandomGenerator.SetSeed(0)

formula = ['8.3*(10^(-3))*(g/v)*((n^3)/((1-n)^2))*(d_10^2)'] 

model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)
distributions = [ot.Uniform(9.8, 9.81), #gravity
                 ot.Uniform(1.16, 4.39), #
                 ot.Uniform(0.34, 0.51), # porosity
                 ot.Uniform(0.09, 2.20)] #grain diameter
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['g', 'v', 'n', 'd_10']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['g', 'v', 'n', 'd_10']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 9.8, b = 9.81), Uniform(a = 1.16, b = 4.39), Uniform(a = 0.34, b = 0.51), Uniform(a = 0.09, b = 2.2), IndependentCopula(dimension = 4))
[-0.0157505,0.0706415,0.17051,0.397052]
The least sensitive variable is:  g
The most sensitive variable is:  d_10
The least sensitive variable is:  g
The most sensitive variable is:  d_10
First-order indices after removing least sensitive variable:  [-0.0747449,0.110556,0.389754,0.411809]
Total order indices after removing least sensitive variable:  [1.83095e-16,0.237924,0.476845,0.886686]
The least sensitive variable is:  g
Modified formula:  ['8.3*(10^(-3))*(0.1/v)*((n^3)/((1-n)^2))*(d_10^2)']


Total Order Indices:  [1.63933e-07,0.192323,0.315303,0.607559]


In [131]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Ueli Schalchli 1992

ot.RandomGenerator.SetSeed(0)

formula = ['K0*(t^(-alpha))'] 

model = ot.SymbolicFunction(['K0', 't', 'alpha'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), #initial hydraulic conductivity
                 ot.Uniform(1, 365), # clogging exponent,
                 ot.Uniform(0.328, 0.935)]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K0', 't', 'alpha']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K0', 't', 'alpha'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['K0', 't', 'alpha']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 1, b = 365), Uniform(a = 0.328, b = 0.935), IndependentCopula(dimension = 3))
[0.151708,0.280101,0.20041]
The least sensitive variable is:  K0
The most sensitive variable is:  t
The least sensitive variable is:  t
The most sensitive variable is:  alpha
First-order indices after removing least sensitive variable:  [-0.0275477,0.410587,0.63914]
Total order indices after removing least sensitive variable:  [-1.50308e-16,0.354396,0.905024]
The least sensitive variable is:  K0
Modified formula:  ['0.1*(t^(-alpha))']


Total Order Indices:  [0.527614,0.410574,0.62494]


In [132]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Grischek et al. 2016

ot.RandomGenerator.SetSeed(0)

formula = ['(k*M*w)^(1/2)'] 

model = ot.SymbolicFunction(['k', 'M', 'w'], formula)
distributions = [ot.Uniform(1e-12, 1e0), #aquifer hydraulic conductivity (m/s)
                 ot.Uniform(0.1, 5145), # saturated thickness of aquifer below the river (m)
                 ot.Uniform(1e-5, 0.002)] # infiltration resistance (s)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['k', 'M', 'w']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['k', 'M', 'w'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['k', 'M', 'w']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 1e-12, b = 1), Uniform(a = 0.1, b = 5145), Uniform(a = 1e-05, b = 0.002), IndependentCopula(dimension = 3))
[0.415293,0.474256,0.307262]
The least sensitive variable is:  w
The most sensitive variable is:  M
The least sensitive variable is:  M
The most sensitive variable is:  w
First-order indices after removing least sensitive variable:  [0.615247,0.317056,-0.153152]
Total order indices after removing least sensitive variable:  [0.84118,0.458767,0]
The least sensitive variable is:  w
Modified formula:  ['(k*M*0.1)^(1/2)']


Total Order Indices:  [0.366026,0.354964,0.389686]


In [133]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Ghani et al. 2017

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*L)/(A*t*h)'] 

model = ot.SymbolicFunction(['Q', 'L', 'A', 't', 'h'], formula)
distributions = [ot.Uniform(0.004992, 840), # volume of discharge (cm3)
                 ot.Uniform(1, 10), # Length of specimen (cm)
                 ot.Uniform(1, 10), # Cross sectional area of permeameter(cm2)
                 ot.Uniform(1, 365), # time for discharge (s)
                 ot.Uniform(0.001, 4.1)] # hydraulic head difference (H1-H2)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'L', 'A', 't', 'h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'L', 'A', 't', 'h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['Q', 'L', 'A', 't', 'h']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 10), Uniform(a = 1, b = 365), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 5))
[-0.0351395,-0.0222249,-0.0260268,0.441476,0.569188]
The least sensitive variable is:  Q
The most sensitive variable is:  h
The least sensitive variable is:  L
The most sensitive variable is:  h
First-order indices after removing least sensitive variable:  [-0.0200566,-0.0288789,-0.00287138,0.014088,0.224229]
Total order indices after removing least sensitive variable:  [-2.04923e-16,0.242211,0.112255,0.210104,0.619158]
The least sensitive variable is:  Q
Modified formula:  ['(0.1*L)/(A*t*h)']


Total Order Indices:  [0.374354,0.0894789,0.396476,0.299372,0.758684]


In [134]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Tao et al. 2019

ot.RandomGenerator.SetSeed(0)

formula = ['(0.05*((U)^5))/(g*((C)^3)*((delta)^2)*D)'] 

model = ot.SymbolicFunction(['U', 'g', 'C', 'delta', 'D'], formula)
distributions = [ot.Uniform(40, 115), # magnitude of depth‐average flow velocity (m/s)
                 ot.Uniform(9.8, 9.81), # acceleration due to gravity (m/s2)
                 ot.Uniform(0.015, 0.0156), # H1/6/n is the Chézy friction coefficient (m1/2/s), H =water depth (m), n = Manning coefficient (s/m1/3)
                 ot.Uniform(0.328, 0.935), # relative density(ρs − ρw)/ρw ρs =sediment density (kg/m3), ρw =water density (kg/m3)
                 ot.Uniform(0.02, 64)] # median grainsize (m/s)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['U', 'g', 'C', 'delta', 'D']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['U', 'g', 'C', 'delta', 'D'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['U', 'g', 'C', 'delta', 'D']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 40, b = 115), Uniform(a = 9.8, b = 9.81), Uniform(a = 0.015, b = 0.0156), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.02, b = 64), IndependentCopula(dimension = 5))
[-0.0221944,-0.0199014,-0.0197839,-0.0109476,0.970109]
The least sensitive variable is:  U
The most sensitive variable is:  D
The least sensitive variable is:  g
The most sensitive variable is:  D
First-order indices after removing least sensitive variable:  [-0.0213139,-0.0212935,-0.0211203,0.00115438,0.886665]
Total order indices after removing least sensitive variable:  [1.48798e-16,4.47595e-08,0.00026183,0.0340394,1.00711]
The least sensitive variable is:  U
Modified formula:  ['(0.05*((0.1)^5))/(g*((C)^3)*((delta)^2)*D)']


Total Order Indices:  [0.179119,8.15096e-08,0.00172876,0.0359562,1.00848]


In [135]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Koren et al. 2021

ot.RandomGenerator.SetSeed(0)

formula = ['((w*S)/(2*T))^(1/2)'] 

model = ot.SymbolicFunction(['w', 'S', 'T'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), # 
                 ot.Uniform(0.1, 50), # coefficient of aquifer elastic storativity
                 ot.Uniform(1860, 2240)] # transmissivity of aquifer
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['w', 'S', 'T']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['w', 'S', 'T'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['w', 'S', 'T']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 0.1, b = 50), Uniform(a = 1860, b = 2240), IndependentCopula(dimension = 3))
[0.469436,0.566234,0.077902]
The least sensitive variable is:  T
The most sensitive variable is:  S
The least sensitive variable is:  T
The most sensitive variable is:  S
First-order indices after removing least sensitive variable:  [0.578695,0.354607,-0.15768]
Total order indices after removing least sensitive variable:  [0.810211,0.499232,-1.15674e-15]
The least sensitive variable is:  T
Modified formula:  ['((w*S)/(2*0.1))^(1/2)']


Total Order Indices:  [0.494611,0.523307,0.00379803]


In [136]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Brunke 1999

ot.RandomGenerator.SetSeed(0)

formula = ['(p*g*R*S)/((ps-p)*g*D)'] 

model = ot.SymbolicFunction(['p', 'g', 'R', 'S', 'ps', 'D'], formula)
distributions = [ot.Uniform(999.8, 1000), # density of water
                 ot.Uniform(9.8, 9.81), # acceleration due to gravity
                 ot.Uniform(1, 5), # hydraulic radius
                 ot.Uniform(0.328, 0.935), # slope
                 ot.Uniform(0.328, 0.935), # density of the sediment
                 ot.Uniform(0.02, 64)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['p', 'g', 'R', 'S', 'ps', 'D']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['p', 'g', 'R', 'S', 'ps', 'D'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['p', 'g', 'R', 'S', 'ps', 'D']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 999.8, b = 1000), Uniform(a = 9.8, b = 9.81), Uniform(a = 1, b = 5), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.02, b = 64), IndependentCopula(dimension = 6))
[-0.0355529,-0.0355529,-0.0326391,-0.0333942,-0.0355563,0.982746]
The least sensitive variable is:  ps
The most sensitive variable is:  D
The least sensitive variable is:  g
The most sensitive variable is:  D
First-order indices after removing least sensitive variable:  [0.0886935,0.0886935,0.128085,0.0792877,0.0886935,0.997527]
Total order indices after removing least sensitive variable:  [-1.85002e-16,1.85002e-16,0.0587889,0.153413,3.70004e-16,0.906278]
The least sensitive variable is:  ps
Modified formula:  ['(p*g*R*S)/((0.1-p)*g*D)']


Total Order Indices:  [9.99423e-16,1.66571e-16,0.00960111,0.00987242,3.58505e-09,1.0341]


In [137]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# Wang et al. 2015

ot.RandomGenerator.SetSeed(0)

formula = ['k0/m0'] 

model = ot.SymbolicFunction(['k0', 'm0'], formula)
distributions = [ot.Uniform(0.017, 7.13), # density of water
                 ot.Uniform(1, 10)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['k0', 'm0']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['k0', 'm0'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['k0', 'm0']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 0.017, b = 7.13), Uniform(a = 1, b = 10), IndependentCopula(dimension = 2))
[0.491051,0.488939]
The least sensitive variable is:  m0
The most sensitive variable is:  k0
The least sensitive variable is:  k0
The most sensitive variable is:  m0
First-order indices after removing least sensitive variable:  [1,0.00547874]
Total order indices after removing least sensitive variable:  [0.994521,3.86498e-16]
The least sensitive variable is:  m0
Modified formula:  ['k0/0.1']


Total Order Indices:  [0.480837,0.662266]


In [138]:
import math
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
ot.Log.Show(ot.Log.NONE)

# cui et al. 2015

ot.RandomGenerator.SetSeed(0)

formula = ['g/(v*8.3e-3*((n^3)/((1-n)^2))*((d)^2))'] 

model = ot.SymbolicFunction(['g', 'v', 'n', 'd'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), # density of water
                 ot.Uniform(0.328, 0.935), # slope
                 ot.Uniform(0.328, 0.935), # density of the sediment
                 ot.Uniform(1, 365)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['g', 'v', 'n', 'd']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['g', 'v', 'n', 'd'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
y = np.array(sensitivityIndices)
mylab=['g', 'v', 'n', 'd']
input_vars = mylab
title_text_size = 25
text_size = 20
tick_label_size = 20
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()


# Print the total order indices
print("Total Order Indices: ", totalOrderIndices)

y = np.array(totalOrderIndices)
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total Order Sensitivity Analysis",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size), range=[-0.5, max(y) + 0.1])
fig.show()

# Plot the first-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getFirstOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="First-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()

# Plot the total-order indices after removing the least sensitive variable
y = np.array(sensitivityAnalysis.getTotalOrderIndices())
fig = go.Figure()
fig.add_trace(go.Bar(x=mylab, y=y, textposition='auto', text=[f"{val:.3f}" for val in y],
                     textfont=dict(size=text_size)))
fig.update_layout(title="Total-order Sensitivity Analysis after removing the least sensitive variable",
                  xaxis_title="Input Variables",
                  yaxis_title="Sobol Indices",
                  title_font=dict(size=title_text_size),
                  xaxis_title_font=dict(size=text_size),
                  yaxis_title_font=dict(size=text_size))
fig.update_xaxes(tickfont=dict(size=tick_label_size))
fig.update_yaxes(tickfont=dict(size=tick_label_size))
fig.show()



# Create a bar trace for the original Sobol indices

y_orig = np.array(sensitivityAnalysis.getFirstOrderIndices())
text_orig = [f"{val:.2f}" for val in y_orig]  # format the values to display on top of the bars
trace_orig = go.Bar(x=input_vars, y=y_orig, name='Original Sobol Indices',
                    text=text_orig, textposition='auto',
                    marker=dict(color='blue'),
                    textfont=dict(size=text_size))

# Create a bar trace for the modified Sobol indices
y_mod = np.array(sensitivityAnalysis.getTotalOrderIndices())
text_mod = [f"{val:.2f}" for val in y_mod]  # format the values to display on top of the bars
trace_mod = go.Bar(x=input_vars, y=y_mod, name='Modified Sobol Indices',
                    text=text_mod, textposition='auto',
                    marker=dict(color='red'),
                    textfont=dict(size=text_size))

# Add annotation to the plot
annotation = [dict(x=0, y=1, xanchor='left', yanchor='bottom',
                   text=f'Least sensitive variable: {leastSensitiveVariable}', showarrow=False),
              dict(x=0, y=0.95, xanchor='left', yanchor='bottom',
                   text=f'Modified formula: {formula}', showarrow=False)]

# Create the layout for the plot
layout = go.Layout(title='Comparison of Original and Modified Sobol Indices',
                   title_font=dict(size=title_text_size),
                   xaxis=dict(title='Input Variables', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   yaxis=dict(title='Sobol Indices', title_font=dict(size=text_size), tickfont=dict(size=tick_label_size)),
                   annotations=annotation)

# Create the figure and add the traces and layout
fig = go.Figure(data=[trace_orig, trace_mod], layout=layout)

# Show the figure
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.328, b = 0.935), Uniform(a = 1, b = 365), IndependentCopula(dimension = 4))
[-0.0326803,-0.036251,-0.0237317,0.0901196]
The least sensitive variable is:  v
The most sensitive variable is:  d
The least sensitive variable is:  g
The most sensitive variable is:  d
First-order indices after removing least sensitive variable:  [-0.0145873,-0.0168394,-0.0199347,0.0399207]
Total order indices after removing least sensitive variable:  [0.00797889,-6.82443e-16,0.470303,0.868935]
The least sensitive variable is:  v
Modified formula:  ['g/(0.1*8.3e-3*((n^3)/((1-n)^2))*((d)^2))']


Total Order Indices:  [0.00078734,0.0418122,0.0757649,1.01973]


In [144]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
# Schulz, Raphael (2020)
ot.Log.Show(ot.Log.NONE)

# Set the random seed
ot.RandomGenerator.SetSeed(0)

# Define the model
formula = ['(c0/(sigma_0*sigma_0))*((theta*theta*theta)/((1-theta)*(1-theta)))']
model = ot.SymbolicFunction(['c0', 'sigma_0', 'theta'], formula)

# Define the distributions
distributions = [ot.Uniform(3.52, 402), # Korenzy constant 
                 ot.Uniform(1, 10), # specific surface with respect to the unit volume of the solid matrix
                 ot.Uniform(-3, 0.99)] #  porosity
distribution = ot.ComposedDistribution(distributions)

# Perform sensitivity analysis
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the first-order indices and identify the most/least sensitive variables
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['c0', 'sigma_0', 'theta']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)
print("First-order indices: ", sensitivityIndices)

# Display the total-order indices and identify the most/least sensitive variables
totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(totalOrderIndices)
maxIndex = np.argmax(totalOrderIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)
print("Total-order indices: ", totalOrderIndices)

# Remove the least sensitive variable and perform sensitivity analysis again
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]
model = ot.SymbolicFunction(['c0', 'sigma_0', 'theta'], formula)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis after removing the least sensitive variable
sensitivityIndicesAfter = sensitivityAnalysis.getFirstOrderIndices()
print("First-order indices after removing least sensitive variable: ", sensitivityIndicesAfter)
print("Total-order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

# Find the minimum and maximum values of the sensitivity indices
y_min = min(min(normalized_values), min(standardized_values))
y_max = max(max(normalized_values), max(standardized_values))

# Define the layout
layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.2 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()


The least sensitive variable is:  sigma_0
The most sensitive variable is:  theta
First-order indices:  [-0.0201689,-0.0203562,0.985809]
The least sensitive variable is:  c0
The most sensitive variable is:  theta
Total-order indices:  [0.0437378,0.645367,1.03409]
First-order indices after removing least sensitive variable:  [0.0129261,0.0660091,0.896859]
Total-order indices after removing least sensitive variable:  [0.0989939,0,0.904372]
Modified formula:  ['(c0/(0.1*0.1))*((theta*theta*theta)/((1-theta)*(1-theta)))']
Normalized first order indices using MinMaxScaler:  [[1.86154008e-04]
 [0.00000000e+00]
 [1.00000000e+00]]
Standardized first order indices using StandardScaler:  [[-0.70690931]
 [-0.70730424]
 [ 1.41421354]]


In [146]:
from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go

ot.Log.Show(ot.Log.NONE)

# Xi-lia Zheng et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['(K0 * ((n*n*n)/((1-n)*(1-n)))*(((1-n0)*(1-n0))/(n0*n0*n0)))/((n*n*n) / (c * ((1 - n)*(1 - n)) * (S*S)))']

model = ot.SymbolicFunction(['K0', 'n', 'n0', 'c', 'S'], formula)
distributions = [ot.Uniform(2.2e-4, 1e-3),  # initial saturated permeability coefficient before recharge during recharge [m/d]
                 ot.Uniform(0.5, 0.99),  # porosity of the sand media [-]
                 ot.Uniform(-3, 0.99),  # initial porosity of aquifer media [-]
                 ot.Uniform(3.52, 402),  # Korenzy constant based on solid volume
                 ot.Uniform(1, 10)]  # Specific surface area based on solid volume
distribution = ot.ComposedDistribution(distributions)

print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K0', 'n', 'n0', 'c', 'S']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K0', 'n', 'n0', 'c', 'S'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.3 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()


ComposedDistribution(Uniform(a = 0.00022, b = 0.001), Uniform(a = 0.5, b = 0.99), Uniform(a = -3, b = 0.99), Uniform(a = 3.52, b = 402), Uniform(a = 1, b = 10), IndependentCopula(dimension = 5))
[-0.00903959,-0.00870381,1,-0.0118258,-0.145612]
The least sensitive variable is:  S
The most sensitive variable is:  n0
The least sensitive variable is:  n
The most sensitive variable is:  n0
First-order indices after removing least sensitive variable:  [0.0151429,0.0121944,0.99627,0.0119104,0.0121944]
Total order indices after removing least sensitive variable:  [0.0128487,0,0.991669,0.401339,-3.0025e-16]
The least sensitive variable is:  S
Modified formula:  ['(K0 * ((n*n*n)/((1-n)*(1-n)))*(((1-n0)*(1-n0))/(n0*n0*n0)))/((n*n*n) / (c * ((1 - n)*(1 - n)) * (0.1*0.1)))']
Normalized first order indices using MinMaxScaler:  [[0.11921357]
 [0.11950667]
 [1.        ]
 [0.11678149]
 [0.        ]]
Standardized first order indices using StandardScaler:  [[-0.41348924]
 [-0.41269131]
 [ 1.98432127]
 [-

In [150]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go

ot.Log.Show(ot.Log.NONE)
# Hubert J. Morel-Seytoux et al., 2016

ot.RandomGenerator.SetSeed(0)
formula = ['(((2*Kh)/(B+H))*(T/((1-(G/Daq))*T)))*(L*(B+H)*(hs-ha))']

model = ot.SymbolicFunction(['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha'], formula)
distributions = [ot.Uniform(0.9144, 457.2), #  horizontal hydraulic conductivity
                 ot.Uniform(10, 250), #  half width of the river
                 ot.Uniform(1.5, 7), # depth of water in river from water surface to streambed
                 ot.Uniform(1e-4, 1e-2), # clogging  layer  correction
                 ot.Uniform(90, 400), # Grid size
                 ot.Uniform(0.1, 5145), # Aquifer thickness
                 ot.Uniform(90, 400), #  length of the river reach within the river cell
                 ot.Uniform(1, 10), # head in river
                 ot.Uniform(130, 270)] #  head in aquifer
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Kh', 'B', 'H', 'T', 'G', 'Daq', 'L', 'hs', 'ha'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())


print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.9144, b = 457.2), Uniform(a = 10, b = 250), Uniform(a = 1.5, b = 7), Uniform(a = 0.0001, b = 0.01), Uniform(a = 90, b = 400), Uniform(a = 0.1, b = 5145), Uniform(a = 90, b = 400), Uniform(a = 1, b = 10), Uniform(a = 130, b = 270), IndependentCopula(dimension = 9))
[0.129918,-0.0164417,-0.0164417,-0.0164417,-0.034778,0.356889,-0.0591219,-0.0165791,0.0248323]
The least sensitive variable is:  L
The most sensitive variable is:  Daq
The least sensitive variable is:  T
The most sensitive variable is:  Daq
First-order indices after removing least sensitive variable:  [0.333992,0.0898557,0.0898557,0.0898557,0.0517256,0.253455,0.0898557,0.0905103,0.121157]
Total order indices after removing least sensitive variable:  [0.184801,-1.81019e-16,-1.81019e-16,-1.81019e-16,0.951163,0.912539,-1.81019e-16,0.000213985,0.0224992]
Modified formula:  ['(((2*Kh)/(B+H))*(T/((1-(G/Daq))*T)))*(0.1*(B+H)*(hs-ha))']
Normalized first order indices using MinMaxScaler:  [[0.4544102

In [151]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

# Rebecca C. Doble et al., 2012

ot.RandomGenerator.SetSeed(0)

formula = ['(Kc(hw + dc))/(dc*dgw*((Sy/tw)+((2*Taq)/(xw*xw))))']

model = ot.SymbolicFunction(['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw'], formula)
distributions = [ot.Uniform(0.017, 7.13), #  Saturated hydraulic conductivity of the surface soil or clogging layer [LT-1]
                 ot.Uniform(10, 250), #  height of the wave above the bankfull elevation
                 ot.Uniform(1.5, 7), # Thickness of the clogging layer
                 ot.Uniform(1e-4, 1e-2), # depth of groundwater = taken at the centre of the flood wave extent
                 ot.Uniform(90, 400), # Aquifer specific yield
                 ot.Uniform(0.1, 5145), # duration of the flood wave
                 ot.Uniform(90, 400), #  Aquifer transmissivity
                 ot.Uniform(130, 270)] # lateral extent of flooding
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Kc', 'hw', 'dc', 'dgw', 'Sy', 'tw', 'Taq', 'xw'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.017, b = 7.13), Uniform(a = 10, b = 250), Uniform(a = 1.5, b = 7), Uniform(a = 0.0001, b = 0.01), Uniform(a = 90, b = 400), Uniform(a = 0.1, b = 5145), Uniform(a = 90, b = 400), Uniform(a = 130, b = 270), IndependentCopula(dimension = 8))
[0.036047,0.033823,0.0700048,0.437864,-0.0160319,0.0676353,0.00989235,0.00268777]
The least sensitive variable is:  Sy
The most sensitive variable is:  dgw
The least sensitive variable is:  xw
The most sensitive variable is:  dgw
First-order indices after removing least sensitive variable:  [0.0301772,-0.0313336,-0.0127174,0.896767,-0.019216,-0.0189167,0.0723795,-0.0317401]
Total order indices after removing least sensitive variable:  [0.140395,0.0167323,0.166773,0.970954,0,2.23466e-05,0.0149032,0.0266576]
The least sensitive variable is:  Sy
Modified formula:  ['(Kc(hw + dc))/(dc*dgw*((0.1/tw)+((2*Taq)/(xw*xw))))']
Normalized first order indices using MinMaxScaler:  [[0.11473765]
 [0.10983772]
 [0.18955173]
 [1.    

In [152]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

# Pedretti et al. 2012

ot.RandomGenerator.SetSeed(0)

formula = ['I0 * (2.7182818285^(-(mp * lambda_z * va * t)))']

model = ot.SymbolicFunction(['I0', 'mp', 'lambda_z', 'va', 't'], formula)
distributions = [ot.Uniform(1e-5, 0.002), # Infiltration capacity [LT-1]
                 ot.Uniform(1, 100), # coefficient of proportionality [-]
                 ot.Uniform(0.044, 0.363), # Filtration coefficient [L-1]
                 ot.Uniform(1e-5, 1.1e-5), # Average particle attachment velocity to the soil matrix
                 ot.Uniform(1, 365)] # time
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['I0', 'mp', 'lambda_z', 'va', 't']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['I0', 'mp', 'lambda_z', 'va', 't'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1e-05, b = 0.002), Uniform(a = 1, b = 100), Uniform(a = 0.044, b = 0.363), Uniform(a = 1e-05, b = 1.1e-05), Uniform(a = 1, b = 365), IndependentCopula(dimension = 5))
[0.999156,0.136031,0.139958,0.13749,0.13731]
The least sensitive variable is:  mp
The most sensitive variable is:  I0
The least sensitive variable is:  va
The most sensitive variable is:  I0
First-order indices after removing least sensitive variable:  [1,-0.204101,-0.204103,-0.204102,-0.204099]
Total order indices after removing least sensitive variable:  [1.20408,-4.59496e-16,1.90881e-09,1.17236e-11,2.81059e-09]
The least sensitive variable is:  mp
Modified formula:  ['I0 * (2.7182818285^(-(0.1 * lambda_z * va * t)))']
Normalized first order indices using MinMaxScaler:  [[1.        ]
 [0.        ]
 [0.00454923]
 [0.00169036]
 [0.00148229]]
Standardized first order indices using StandardScaler:  [[ 1.99998639]
 [-0.50483208]
 [-0.49343708]
 [-0.50059803]
 [-0.50111921]]


In [153]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Jung-Woo Kim et al., 2009

ot.RandomGenerator.SetSeed(0)

formula = ['((T*(4*3.141592654*delta_h))/2.3)/(delta_h/(Lt*A))'] 

model = ot.SymbolicFunction(['T', 'delta_h', 'Lt', 'A'], formula)
distributions = [ot.Uniform(1860, 2240), # Transmissivity
                 ot.Uniform(0.001, 4.1), # hydraulic head loss [mm]
                 ot.Uniform(130, 270),# length of the glass bead layer [mm]
                 ot.Uniform(90, 400)] # cross-sectional area of the glass bead layer [mm2]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['T', 'delta_h', 'Lt', 'A']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['T', 'delta_h', 'Lt', 'A'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1860, b = 2240), Uniform(a = 0.001, b = 4.1), Uniform(a = 130, b = 270), Uniform(a = 90, b = 400), IndependentCopula(dimension = 4))
[0.0648774,0.0618126,0.248268,0.585619]
The least sensitive variable is:  delta_h
The most sensitive variable is:  A
The least sensitive variable is:  delta_h
The most sensitive variable is:  A
First-order indices after removing least sensitive variable:  [-0.00576555,-0.0209149,0.373041,0.652473]
Total order indices after removing least sensitive variable:  [0.0179644,-3.36285e-16,0.314736,0.70832]
The least sensitive variable is:  delta_h
Modified formula:  ['((T*(4*3.141592654*0.1))/2.3)/(0.1/(Lt*A))']
Normalized first order indices using MinMaxScaler:  [[0.00585106]
 [0.        ]
 [0.35596311]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.82180073]
 [-0.83617124]
 [ 0.03809229]
 [ 1.61987968]]


In [165]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Jena-Olaf Delfs 2011

ot.RandomGenerator.SetSeed(0)

formula = ['delta_1*(2.71828128*(delta_2*(h-hg)))'] 

model = ot.SymbolicFunction(['delta_1', 'delta_2', 'h', 'hg'], formula)
distributions = [ot.Uniform(1e-1, 1), # Λ1 and Λ2 are parameters and implemented into OGS code.
                 ot.Uniform(0.005, 0.1), 
                 ot.Uniform(1, 5), # h = H + b the water height in the river
                 ot.Uniform(1, 10)] # water table height
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['delta_1', 'delta_2', 'h', 'hg']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['delta_1', 'delta_2', 'h', 'hg'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.8, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 1), Uniform(a = 0.005, b = 0.1), Uniform(a = 1, b = 5), Uniform(a = 1, b = 10), IndependentCopula(dimension = 4))
[0.091938,0.252023,0.254644,0.362292]
The least sensitive variable is:  delta_1
The most sensitive variable is:  hg
The least sensitive variable is:  delta_1
The most sensitive variable is:  hg
First-order indices after removing least sensitive variable:  [0.150803,0.410453,0.230992,0.586284]
Total order indices after removing least sensitive variable:  [1.96796e-16,0.388793,0.149777,0.524644]
The least sensitive variable is:  delta_1
Modified formula:  ['0.1*(2.71828128*(delta_2*(h-hg)))']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.59212962]
 [0.60182585]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-1.53690248]
 [ 0.12228431]
 [ 0.14945379]
 [ 1.26516437]]


In [155]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Xuezhen Zhang et al., 2020

ot.RandomGenerator.SetSeed(0)

formula = ['(Et0 - Et)/Et0'] 

model = ot.SymbolicFunction(['Et0', 'Et'], formula)
distributions = [ot.Uniform(0.143, 1.194), # initial water absorbing capacity
                 ot.Uniform(6.056, 6.282)] # water absorbing capacity at time t
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Et0', 'Et']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]



print("The least sensitive variable is: ", leastSensitiveVariable)

# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.143, b = 1.194), Uniform(a = 6.056, b = 6.282), IndependentCopula(dimension = 2))
[0.999656,-0.0355463]
The least sensitive variable is:  Et
The most sensitive variable is:  Et0
The least sensitive variable is:  Et
The most sensitive variable is:  Et0
The least sensitive variable is:  Et
Normalized first order indices using MinMaxScaler:  [[1.]
 [0.]]
Standardized first order indices using StandardScaler:  [[ 1.]
 [-1.]]


In [156]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
#Xinqiang Du et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*l)/(3.141592654*(r^2)*d_h)'] 

model = ot.SymbolicFunction(['Q', 'l', 'r', 'd_h'], formula)
distributions = [ot.Uniform(0.004992, 840), # flow rate [m3d-1]
                 ot.Uniform(1, 10), # distance between any two piezometric tubes along the column [m]
                 ot.Uniform(1, 5), # Inner diameter of the column [m]
                 ot.Uniform(0.001, 4.1)] #  hydraulic head difference at a difference l [m]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'l', 'r', 'd_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'l', 'r', 'd_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.0591762,0.0333257,0.0419325,0.368628]
The least sensitive variable is:  l
The most sensitive variable is:  d_h
The least sensitive variable is:  Q
The most sensitive variable is:  d_h
First-order indices after removing least sensitive variable:  [0.0123108,-0.00530018,0.0374737,0.298991]
Total order indices after removing least sensitive variable:  [0.03271,0,0.213422,0.761323]
The least sensitive variable is:  l
Modified formula:  ['(Q*0.1)/(3.141592654*(r^2)*d_h)']
Normalized first order indices using MinMaxScaler:  [[0.07709627]
 [0.        ]
 [0.02566887]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.47386044]
 [-0.65781708]
 [-0.59656951]
 [ 1.72824703]]


In [157]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Kristin J. Rehg et al., 2004

ot.RandomGenerator.SetSeed(0)

formula = ['d/((m*theta)+d)'] 

model = ot.SymbolicFunction(['d', 'm', 'theta'], formula)
distributions = [ot.Uniform(1.5, 10.87), # effective stream depth (total volume of stream water per unit bed area)
                 ot.Uniform(0, 0.18), # average depth of solute penetration into the bed
                 ot.Uniform(-3, 0.99)] # bulk porosity
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['d', 'm', 'theta']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['d', 'm', 'theta'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.2, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1.5, b = 10.87), Uniform(a = 0, b = 0.18), Uniform(a = -3, b = 0.99), IndependentCopula(dimension = 3))
[-0.128585,-0.0601137,0.3872]
The least sensitive variable is:  d
The most sensitive variable is:  theta
The least sensitive variable is:  d
The most sensitive variable is:  theta
First-order indices after removing least sensitive variable:  [-0.0493395,-0.00470377,0.0421137]
Total order indices after removing least sensitive variable:  [2.05753e-16,0.946556,1.08424]
The least sensitive variable is:  d
Modified formula:  ['0.1/((m*theta)+0.1)']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.13275118]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.85148965]
 [-0.55212226]
 [ 1.40361191]]


In [163]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# E. Hoehn et al., 2006

ot.RandomGenerator.SetSeed(0)

formula = ['(x*ne)/tao_w'] 

model = ot.SymbolicFunction(['x', 'ne', 'tao_w'], formula)
distributions = [ot.Uniform(1, 10), # path length from the infiltration point in the river to a nearby monitoring well
                 ot.Uniform(-3, 0.99), # flow-effective porosity of the riverbed and aquifer materials
                 ot.Uniform(1, 365)] # mean residence time of freshly infiltrated water
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['x', 'ne', 'tao_w']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['x', 'ne', 'tao_w'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1, b = 10), Uniform(a = -3, b = 0.99), Uniform(a = 1, b = 365), IndependentCopula(dimension = 3))
[-0.0128778,0.0191238,-0.391791]
The least sensitive variable is:  tao_w
The most sensitive variable is:  ne
The least sensitive variable is:  ne
The most sensitive variable is:  tao_w
First-order indices after removing least sensitive variable:  [-0.0155298,0.703508,-0.0118076]
Total order indices after removing least sensitive variable:  [0.336184,0.802738,5.93729e-16]
The least sensitive variable is:  tao_w
Modified formula:  ['(x*ne)/0.1']
Normalized first order indices using MinMaxScaler:  [[0.92212126]
 [1.        ]
 [0.        ]]
Standardized first order indices using StandardScaler:  [[ 0.61963314]
 [ 0.79111081]
 [-1.41074394]]


In [166]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
#Konstantinos N. Moutsopoulos 2013

ot.RandomGenerator.SetSeed(0)

formula = ['(K*b)/k'] 

model = ot.SymbolicFunction(['K', 'b', 'k'], formula)
distributions = [ot.Uniform(1e-12, 1e0), # hydraulic conductivity of the aquifer 
                 ot.Uniform(1, 10), # length of the base of semipervious layer of rectangular shape
                 ot.Uniform(0.1, 50)] # the hydraulic conductivity of the clogging layer (m/d).
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K', 'b', 'k']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K', 'b', 'k'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1e-12, b = 1), Uniform(a = 1, b = 10), Uniform(a = 0.1, b = 50), IndependentCopula(dimension = 3))
[-0.0438464,-0.0496101,0.564772]
The least sensitive variable is:  b
The most sensitive variable is:  k
The least sensitive variable is:  b
The most sensitive variable is:  k
First-order indices after removing least sensitive variable:  [-0.0279695,-0.0697694,0.891142]
Total order indices after removing least sensitive variable:  [0.634342,-3.39972e-16,1.03177]
The least sensitive variable is:  b
Modified formula:  ['(K*0.1)/k']
Normalized first order indices using MinMaxScaler:  [[0.00938127]
 [0.        ]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.69708633]
 [-0.71708012]
 [ 1.41416645]]


In [167]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Sebastian Rudnick et al., 2014

ot.RandomGenerator.SetSeed(0)

formula = ['1/(G*(1/(1+(2.718281828^(-k*G*norm))*((G/k0)-1)))*clogg_max)'] 

model = ot.SymbolicFunction(['G', 'k', 'norm', 'k0', 'clogg_max'], formula) # G = 1, k = 12, and k0 = 0.02
distributions = [ot.Uniform(0.9, 1), # soil heat-flux density (MJ/[m2*d]),
                 ot.Uniform(1, 12), # hydraulic gradients
                 ot.Uniform(-0.26, -0.25), # vertical hydraulic gradient of the cell normalized by the steepest negative gradient of −0.25 m/m
                 ot.Uniform(0.01, 0.02), # hydraulic gradient at 0
                 ot.Uniform(1, 100)] # cloggmax is the maximum assumed clogging factor of 100 (–)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['G', 'k', 'norm', 'k0', 'clogg_max']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['G', 'k', 'norm', 'k0', 'clogg_max'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.9, b = 1), Uniform(a = 1, b = 12), Uniform(a = -0.26, b = -0.25), Uniform(a = 0.01, b = 0.02), Uniform(a = 1, b = 100), IndependentCopula(dimension = 5))
[-0.00828519,0.0505858,-0.00702523,0.0028972,0.57688]
The least sensitive variable is:  G
The most sensitive variable is:  clogg_max
The least sensitive variable is:  norm
The most sensitive variable is:  clogg_max
First-order indices after removing least sensitive variable:  [-0.0275833,-0.0356542,-0.0274681,-0.00769843,0.981265]
Total order indices after removing least sensitive variable:  [0,0.0071551,1.78227e-06,0.017287,1.01591]
The least sensitive variable is:  G
Modified formula:  ['1/(0.1*(1/(1+(2.718281828^(-k*0.1*norm))*((0.1/k0)-1)))*clogg_max)']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.1006058 ]
 [0.00215316]
 [0.0191098 ]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.57597149]
 [-0.31771458]
 [-0.57044429]
 [-0.5269163 ]
 [ 1.991

In [168]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Vassilios Kaleris 1998

ot.RandomGenerator.SetSeed(0)

formula = ['((a-psi_ij)*A)/(RL_ij)'] 

model = ot.SymbolicFunction(['a', 'psi_ij', 'A', 'RL_ij'], formula)
distributions = [ot.Uniform(1.5, 7), # water depth in the stream [m]
                 ot.Uniform(1, 2), # pressure head below the clogging layer [m]
                 ot.Uniform(90, 400), # stream bed area corresponding to the node (i j) = [m2]
                 ot.Uniform(1e-12, 1e0)] # hydraulic resistance of the clogging layer at the node (i, j) = [s-1]

distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['a', 'psi_ij', 'A', 'RL_ij']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['a', 'psi_ij', 'A', 'RL_ij'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()
layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1.5, b = 7), Uniform(a = 1, b = 2), Uniform(a = 90, b = 400), Uniform(a = 1e-12, b = 1), IndependentCopula(dimension = 4))
[0.108492,0.117599,0.113425,0.71928]
The least sensitive variable is:  a
The most sensitive variable is:  RL_ij
The least sensitive variable is:  psi_ij
The most sensitive variable is:  RL_ij
First-order indices after removing least sensitive variable:  [-0.0109299,-0.0040626,0.00217798,0.967399]
Total order indices after removing least sensitive variable:  [-9.28879e-16,0.0862424,0.0681294,1.02027]
The least sensitive variable is:  a
Modified formula:  ['((0.1-psi_ij)*A)/(RL_ij)']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.01490968]
 [0.0080771 ]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.59513725]
 [-0.56044156]
 [-0.57634138]
 [ 1.73192019]]


In [169]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
#M. Dimkic et al., 2011

ot.RandomGenerator.SetSeed(0)

formula = ['(K_l/d)*Delta_S*2*3.141592654*r*L'] 

model = ot.SymbolicFunction(['K_l', 'd', 'Delta_S', 'r', 'L'], formula)
distributions = [ot.Uniform(0.1, 50), #  Representative Values of the Coefficient of Local Hydraulic Losses at Well Laterals, KL /d conductivity of the well-lateral colmated layer [L T−1]
                 ot.Uniform(0.1, 0.11), # thickness of the colmated layer [L]
                 ot.Uniform(0.001, 4.1), # piezometric head drop at the colmated layer, the local drawdown
                 ot.Uniform(1, 5), # radius of the well lateral [L]
                 ot.Uniform(1, 10)] # length of the well lateral [L]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K_l', 'd', 'Delta_S', 'r', 'L']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K_l', 'd', 'Delta_S', 'r', 'L'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()
layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 50), Uniform(a = 0.1, b = 0.11), Uniform(a = 0.001, b = 4.1), Uniform(a = 1, b = 5), Uniform(a = 1, b = 10), IndependentCopula(dimension = 5))
[0.329926,0.0994834,0.19751,0.133271,0.209995]
The least sensitive variable is:  d
The most sensitive variable is:  K_l
The least sensitive variable is:  d
The most sensitive variable is:  L
First-order indices after removing least sensitive variable:  [0.151239,-0.0734726,0.220595,0.0603313,0.0538236]
Total order indices after removing least sensitive variable:  [0.660711,1.12872e-16,0.617057,0.312157,0.315331]
The least sensitive variable is:  d
Modified formula:  ['(K_l/0.1)*Delta_S*2*3.141592654*r*L']
Normalized first order indices using MinMaxScaler:  [[1.        ]
 [0.        ]
 [0.42538179]
 [0.14662221]
 [0.47956177]]
Standardized first order indices using StandardScaler:  [[ 1.71571039]
 [-1.19381761]
 [ 0.04384261]
 [-0.76721619]
 [ 0.2014808 ]]


In [170]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

# Hong Hanh Nguyen et al., 2021

ot.RandomGenerator.SetSeed(0)

formula = ['h*(k_c/B)*(W_p*L*p_leak)'] 

model = ot.SymbolicFunction(['h', 'k_c', 'B', 'W_p', 'L', 'p_leak'], formula)
distributions = [ot.Uniform(0, 0.1), # wastewater level in the pipe (L)
                 ot.Uniform(0.1, 50), # hydraulic conductivity of colmation layer (L T−1)
                 ot.Uniform(0.1, 0.11), # thickness of the colmation layer (L)
                 ot.Uniform(99.9, 99.999), # wetted perimeter generated with the water level in the sewer (L)
                 ot.Uniform(1, 10), # length of the sewer segment of interest (L)
                 ot.Uniform(0.005, 0.1)] # percentage of leaking surfaces from the total area under the wetted perimeter.
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['h', 'k_c', 'B', 'W_p', 'L', 'p_leak']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['h', 'k_c', 'B', 'W_p', 'L', 'p_leak'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0, b = 0.1), Uniform(a = 0.1, b = 50), Uniform(a = 0.1, b = 0.11), Uniform(a = 99.9, b = 99.999), Uniform(a = 1, b = 10), Uniform(a = 0.005, b = 0.1), IndependentCopula(dimension = 6))
[0.268363,0.182783,-0.00227884,-0.00993451,0.0387578,0.116153]
The least sensitive variable is:  W_p
The most sensitive variable is:  h
The least sensitive variable is:  W_p
The most sensitive variable is:  h
First-order indices after removing least sensitive variable:  [0.503517,0.454298,0.345984,0.329781,0.375093,0.439567]
Total order indices after removing least sensitive variable:  [0.321005,0.324899,0.0010441,2.97042e-16,0.19395,0.29791]
The least sensitive variable is:  W_p
Modified formula:  ['h*(k_c/B)*(0.1*L*p_leak)']
Normalized first order indices using MinMaxScaler:  [[1.        ]
 [0.69248833]
 [0.02750893]
 [0.        ]
 [0.17496483]
 [0.45306699]]
Standardized first order indices using StandardScaler:  [[ 1.66970299]
 [ 0.82612592]
 [-0.99806949]
 [-1.073532

In [171]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)

# Mohamed Amine Boukhemacha et al., 2015

ot.RandomGenerator.SetSeed(0)

formula = ['K*(delta_H/B)*(W_p*L*p_leaks)'] 

model = ot.SymbolicFunction(['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks'], formula)
distributions = [ot.Uniform(0.1, 50), # hydraulic conductivity of the clogging layer [m/day] = 0.15 – 33.09 (literature) = 7.34 – 29.37 (NEIMO calibration)
                 ot.Uniform(0, 0.1), # hydraulic head difference between the water level in the sewer and the hydraulic head of the surrounding groundwater (taken to be null) [m]
                 ot.Uniform(0.1, 0.11), # thickness of the clogging layer [m] = 0.1
                 ot.Uniform(99.9, 99.999), # wetted parameter
                 ot.Uniform(1, 10), # length
                 ot.Uniform(0.005, 0.1)] # percentage of the leaky area from the total area generated by the wetted perimeter
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K', 'delta_H', 'B', 'W_p', 'L', 'p_leaks'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.1, b = 50), Uniform(a = 0, b = 0.1), Uniform(a = 0.1, b = 0.11), Uniform(a = 99.9, b = 99.999), Uniform(a = 1, b = 10), Uniform(a = 0.005, b = 0.1), IndependentCopula(dimension = 6))
[0.266466,0.184559,-0.00222142,-0.00990753,0.0392449,0.116777]
The least sensitive variable is:  W_p
The most sensitive variable is:  K
The least sensitive variable is:  W_p
The most sensitive variable is:  K
First-order indices after removing least sensitive variable:  [0.502451,0.455119,0.345814,0.329618,0.374831,0.439457]
Total order indices after removing least sensitive variable:  [0.318892,0.327085,0.00104319,-2.97259e-16,0.194025,0.297932]
The least sensitive variable is:  W_p
Modified formula:  ['K*(delta_H/B)*(0.1*L*p_leaks)']
Normalized first order indices using MinMaxScaler:  [[1.        ]
 [0.70363702]
 [0.02781061]
 [0.        ]
 [0.17784797]
 [0.45838368]]
Standardized first order indices using StandardScaler:  [[ 1.65450442]
 [ 0.84455306]
 [-1.00246082]
 [

In [172]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Stephane Descloux et al., 2010

ot.RandomGenerator.SetSeed(0)

formula = ['(((Mt-sum_Mw)*p)/Vt)*100'] 

model = ot.SymbolicFunction(['Mt', 'sum_Mw', 'p', 'Vt'], formula)
distributions = [ot.Uniform(1, 10), # Total mass of sample core [kg]
                 ot.Uniform(1, 5), # Mass of wet sediments
                 ot.Uniform(999.9, 1000), # Water density [kg/m3] = 1000
                 ot.Uniform(1, 10)] #  Total volume of sample core
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Mt', 'sum_Mw', 'p', 'Vt']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Mt', 'sum_Mw', 'p', 'Vt'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()
layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 999.9, b = 1000), Uniform(a = 1, b = 10), IndependentCopula(dimension = 4))
[0.534418,0.0945905,0.0528694,0.3402]
The least sensitive variable is:  p
The most sensitive variable is:  Mt
The least sensitive variable is:  p
The most sensitive variable is:  Vt
First-order indices after removing least sensitive variable:  [0.499166,0.125385,0.10839,0.33692]
Total order indices after removing least sensitive variable:  [0.427536,0.118886,3.98241e-16,0.457101]
The least sensitive variable is:  p
Modified formula:  ['(((Mt-sum_Mw)*0.1)/Vt)*100']
Normalized first order indices using MinMaxScaler:  [[1.        ]
 [0.08663951]
 [0.        ]
 [0.59667974]]
Standardized first order indices using StandardScaler:  [[ 1.43109482]
 [-0.82576421]
 [-1.03984524]
 [ 0.43451463]]


In [173]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# K. Pholkern et al., 2015

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*L)/(A*delta_h)'] 

model = ot.SymbolicFunction(['Q', 'L', 'A', 'delta_h'], formula)
distributions = [ot.Uniform(0.004992, 840), # Outflow / Infiltration discharge [m3/s]
                 ot.Uniform(0.003, 0.006), #Thickness of clogging layer
                 ot.Uniform(5.65, 237.56), # Area of flow [m2]
                 ot.Uniform(0.001, 4.1)] #  Head difference [m]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'L', 'A', 'delta_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'L', 'A', 'delta_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 0.003, b = 0.006), Uniform(a = 5.65, b = 237.56), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.020594,0.0107956,0.0329896,0.267529]
The least sensitive variable is:  L
The most sensitive variable is:  delta_h
The least sensitive variable is:  Q
The most sensitive variable is:  delta_h
First-order indices after removing least sensitive variable:  [0.00593693,-0.0129998,0.0768694,0.284839]
Total order indices after removing least sensitive variable:  [0.0508459,4.32186e-16,0.141652,0.802564]
The least sensitive variable is:  L
Modified formula:  ['(Q*0.1)/(A*delta_h)']
Normalized first order indices using MinMaxScaler:  [[0.03816549]
 [0.        ]
 [0.08644736]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.583887  ]
 [-0.67559671]
 [-0.46786815]
 [ 1.72735186]]


In [174]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Jonathan A. Czuba et al., 2021

ot.RandomGenerator.SetSeed(0)

formula = ['((As/(R*tc))^(1/18))*(((Qb*g)/(a*kn*Bb))^(1/3))*(S^(7/18))'] 

model = ot.SymbolicFunction(['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S'], formula)
distributions = [ot.Uniform(0.5, 0.6), # constant
                 ot.Uniform(0.6, 1.65), #Submurged specific gravity of sediment
                 ot.Uniform(0.03, 0.6), # dimensionless critical bed shear stress (or Shields parameter) for initiation of motion
                 ot.Uniform(0.004992, 840), # denotes that these  values are at the specific flow condition when Dx is mobilized.
                 ot.Uniform(9.8, 9.81, ), # gravity
                 ot.Uniform(3, 3.3), # the coefficient of the Manning–Strickler form of the logarithmic law for estimating the velocity distribution in the vertical direction
                 ot.Uniform(1, 3.3), # conversion factor between SI and English units
                 ot.Uniform(3, 3.3), # channel width
                 ot.Uniform(3.5, 5)] #  Channel slope
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['As', 'R', 'tc', 'Qb', 'g', 'a', 'kn', 'Bb', 'S'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.5, b = 0.6), Uniform(a = 0.6, b = 1.65), Uniform(a = 0.03, b = 0.6), Uniform(a = 0.004992, b = 840), Uniform(a = 9.8, b = 9.81), Uniform(a = 3, b = 3.3), Uniform(a = 1, b = 3.3), Uniform(a = 3, b = 3.3), Uniform(a = 3.5, b = 5), IndependentCopula(dimension = 9))
[0.105732,0.106207,0.154237,0.712589,0.105891,0.103065,0.255798,0.112279,0.119917]
The least sensitive variable is:  a
The most sensitive variable is:  Qb
The least sensitive variable is:  g
The most sensitive variable is:  Qb
First-order indices after removing least sensitive variable:  [-0.0417975,-0.033906,-0.0107761,0.783356,-0.0420187,-0.042045,0.0627181,-0.0395349,-0.0348003]
Total order indices after removing least sensitive variable:  [0.00012134,0.00352525,0.0217879,0.807672,1.06212e-07,-1.61415e-16,0.162021,0.000824283,0.0166113]
The least sensitive variable is:  a
Modified formula:  ['((As/(R*tc))^(1/18))*(((Qb*g)/(0.1*kn*Bb))^(1/3))*(S^(7/18))']
Normalized first order indices using

In [175]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# STEPHANIE RINCK-PFEIFFER et al., 1999

ot.RandomGenerator.SetSeed(0)

formula = ['Q*(delta_x/(3.141592654*(r^2)))*delta_h'] 

model = ot.SymbolicFunction(['Q', 'delta_x', 'r', 'delta_h'], formula)
distributions = [ot.Uniform(0.004992, 840), #m3/d, Flowrate
                 ot.Uniform(1, 10), #m length of the column
                 ot.Uniform(1, 5), #m radius
                 ot.Uniform(0.001, 4.1)] #m hydraulic head difference
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'delta_x', 'r', 'delta_h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'delta_x', 'r', 'delta_h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 5), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 4))
[0.0668982,0.0425869,0.579263,0.00201209]
The least sensitive variable is:  delta_h
The most sensitive variable is:  r
The least sensitive variable is:  Q
The most sensitive variable is:  r
First-order indices after removing least sensitive variable:  [0.0137541,0.108376,0.394785,-0.0370068]
Total order indices after removing least sensitive variable:  [0.154702,0.226437,0.83255,-3.84768e-16]
The least sensitive variable is:  delta_h
Modified formula:  ['Q*(delta_x/(3.141592654*(r^2)))*0.1']
Normalized first order indices using MinMaxScaler:  [[0.1124055 ]
 [0.07028978]
 [1.        ]
 [0.        ]]
Standardized first order indices using StandardScaler:  [[-0.44850471]
 [-0.55157268]
 [ 1.72366726]
 [-0.72358986]]


In [176]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Ali rifat cabalar 2016

ot.RandomGenerator.SetSeed(0)

formula = ['6*(10^(-4))*(g/v)*(1+10*(n-0.026))*(d_10^2)'] 

model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)
distributions = [ot.Uniform(9.8, 9.81),
                 ot.Uniform(1.16, 4.39),
                 ot.Uniform(0.34, 0.47),
                 ot.Uniform(0.09, 2.20)]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['g', 'v', 'n', 'd_10']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['g', 'v', 'n', 'd_10'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 9.8, b = 9.81), Uniform(a = 1.16, b = 4.39), Uniform(a = 0.34, b = 0.47), Uniform(a = 0.09, b = 2.2), IndependentCopula(dimension = 4))
[-0.0819419,0.123099,-0.0883531,0.650553]
The least sensitive variable is:  n
The most sensitive variable is:  d_10
The least sensitive variable is:  g
The most sensitive variable is:  d_10
First-order indices after removing least sensitive variable:  [0.0682256,0.357191,0.0683116,0.732095]
Total order indices after removing least sensitive variable:  [2.01463e-07,0.33072,3.86112e-16,0.842268]
The least sensitive variable is:  n
Modified formula:  ['6*(10^(-4))*(g/v)*(1+10*(0.1-0.026))*(d_10^2)']
Normalized first order indices using MinMaxScaler:  [[0.00867662]
 [0.28616856]
 [0.        ]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.773915  ]
 [-0.09222758]
 [-0.79523001]
 [ 1.66137259]]


In [177]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Ueli Schalchli 1992

ot.RandomGenerator.SetSeed(0)

formula = ['K0*(t^(-alpha))'] 

model = ot.SymbolicFunction(['K0', 't', 'alpha'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), #initial hydraulic conductivity
                 ot.Uniform(0.328, 0.935), # clogging exponent,
                 ot.Uniform(1, 365)]
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['K0', 't', 'alpha']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['K0', 't', 'alpha'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()
layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 0.328, b = 0.935), Uniform(a = 1, b = 365), IndependentCopula(dimension = 3))
[-0.01,0,-0.0100025]
The least sensitive variable is:  alpha
The most sensitive variable is:  t
The least sensitive variable is:  K0
The most sensitive variable is:  alpha
First-order indices after removing least sensitive variable:  [0.997421,-0.119085,-0.117339]
Total order indices after removing least sensitive variable:  [1.11416,0.00286427,-1.89841e-16]
The least sensitive variable is:  alpha
Modified formula:  ['K0*(t^(-0.1))']
Normalized first order indices using MinMaxScaler:  [[2.4988732e-04]
 [1.0000000e+00]
 [0.0000000e+00]]
Standardized first order indices using StandardScaler:  [[-0.70684169]
 [ 1.41421353]
 [-0.70737184]]


In [178]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Grischek et al. 2016

ot.RandomGenerator.SetSeed(0)

formula = ['(k*M*w)^(1/2)'] 

model = ot.SymbolicFunction(['k', 'M', 'w'], formula)
distributions = [ot.Uniform(1e-12, 1e0), #aquifer hydraulic conductivity (m/s)
                 ot.Uniform(0.1, 5145), # saturated thickness of aquifer below the river (m)
                 ot.Uniform(1e-5, 0.002)] # infiltration resistance (s)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['k', 'M', 'w']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['k', 'M', 'w'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 1e-12, b = 1), Uniform(a = 0.1, b = 5145), Uniform(a = 1e-05, b = 0.002), IndependentCopula(dimension = 3))
[0.415293,0.474256,0.307262]
The least sensitive variable is:  w
The most sensitive variable is:  M
The least sensitive variable is:  M
The most sensitive variable is:  w
First-order indices after removing least sensitive variable:  [0.615247,0.317056,-0.153152]
Total order indices after removing least sensitive variable:  [0.84118,0.458767,0]
The least sensitive variable is:  w
Modified formula:  ['(k*M*0.1)^(1/2)']
Normalized first order indices using MinMaxScaler:  [[0.64691736]
 [1.        ]
 [0.        ]]
Standardized first order indices using StandardScaler:  [[ 0.23653549]
 [ 1.08922479]
 [-1.32576028]]


In [179]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Ghani et al. 2017

ot.RandomGenerator.SetSeed(0)

formula = ['(Q*L)/(A*t*h)'] 

model = ot.SymbolicFunction(['Q', 'L', 'A', 't', 'h'], formula)
distributions = [ot.Uniform(0.004992, 840), # volume of discharge (cm3)
                 ot.Uniform(1, 10), # Length of specimen (cm)
                 ot.Uniform(1, 10), # Cross sectional area of permeameter(cm2)
                 ot.Uniform(1, 365), # time for discharge (s)
                 ot.Uniform(0.001, 4.1)] # hydraulic head difference (H1-H2)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['Q', 'L', 'A', 't', 'h']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['Q', 'L', 'A', 't', 'h'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.004992, b = 840), Uniform(a = 1, b = 10), Uniform(a = 1, b = 10), Uniform(a = 1, b = 365), Uniform(a = 0.001, b = 4.1), IndependentCopula(dimension = 5))
[-0.0351395,-0.0222249,-0.0260268,0.441476,0.569188]
The least sensitive variable is:  Q
The most sensitive variable is:  h
The least sensitive variable is:  L
The most sensitive variable is:  h
First-order indices after removing least sensitive variable:  [-0.0200566,-0.0288789,-0.00287138,0.014088,0.224229]
Total order indices after removing least sensitive variable:  [-2.04923e-16,0.242211,0.112255,0.210104,0.619158]
The least sensitive variable is:  Q
Modified formula:  ['(0.1*L)/(A*t*h)']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.02137021]
 [0.01507902]
 [0.78867029]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.83458396]
 [-0.78572351]
 [-0.80010756]
 [ 0.96861733]
 [ 1.45179769]]


In [180]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Tao et al. 2019

ot.RandomGenerator.SetSeed(0)

formula = ['(0.05*((U)^5))/(g*((C)^3)*((delta)^2)*D)'] 

model = ot.SymbolicFunction(['U', 'g', 'C', 'delta', 'D'], formula)
distributions = [ot.Uniform(40, 115), # magnitude of depth‐average flow velocity (m/s)
                 ot.Uniform(9.8, 9.81), # acceleration due to gravity (m/s2)
                 ot.Uniform(0.015, 0.0156), # H1/6/n is the Chézy friction coefficient (m1/2/s), H =water depth (m), n = Manning coefficient (s/m1/3)
                 ot.Uniform(0.328, 0.935), # relative density(ρs − ρw)/ρw ρs =sediment density (kg/m3), ρw =water density (kg/m3)
                 ot.Uniform(0.02, 64)] # median grainsize (m/s)
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['U', 'g', 'C', 'delta', 'D']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['U', 'g', 'C', 'delta', 'D'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 40, b = 115), Uniform(a = 9.8, b = 9.81), Uniform(a = 0.015, b = 0.0156), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.02, b = 64), IndependentCopula(dimension = 5))
[-0.0221944,-0.0199014,-0.0197839,-0.0109476,0.970109]
The least sensitive variable is:  U
The most sensitive variable is:  D
The least sensitive variable is:  g
The most sensitive variable is:  D
First-order indices after removing least sensitive variable:  [-0.0213139,-0.0212935,-0.0211203,0.00115438,0.886665]
Total order indices after removing least sensitive variable:  [1.48798e-16,4.47595e-08,0.00026183,0.0340394,1.00711]
The least sensitive variable is:  U
Modified formula:  ['(0.05*((0.1)^5))/(g*((C)^3)*((delta)^2)*D)']
Normalized first order indices using MinMaxScaler:  [[0.        ]
 [0.00231079]
 [0.00242921]
 [0.01133411]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.51006272]
 [-0.5042627 ]
 [-0.50396547]
 [-0.48161447]
 [ 1.99990536]]


In [181]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Koren et al. 2021

ot.RandomGenerator.SetSeed(0)

formula = ['((w*S)/(2*T))^(1/2)'] 

model = ot.SymbolicFunction(['w', 'S', 'T'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), # 
                 ot.Uniform(0.1, 50), # coefficient of aquifer elastic storativity
                 ot.Uniform(1860, 2240)] # transmissivity of aquifer
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['w', 'S', 'T']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['w', 'S', 'T'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 0.1, b = 50), Uniform(a = 1860, b = 2240), IndependentCopula(dimension = 3))
[0.469436,0.566234,0.077902]
The least sensitive variable is:  T
The most sensitive variable is:  S
The least sensitive variable is:  T
The most sensitive variable is:  S
First-order indices after removing least sensitive variable:  [0.578695,0.354607,-0.15768]
Total order indices after removing least sensitive variable:  [0.810211,0.499232,-1.15674e-15]
The least sensitive variable is:  T
Modified formula:  ['((w*S)/(2*0.1))^(1/2)']
Normalized first order indices using MinMaxScaler:  [[0.80177839]
 [1.        ]
 [0.        ]]
Standardized first order indices using StandardScaler:  [[ 0.46535782]
 [ 0.92385971]
 [-1.38921753]]


In [182]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Brunke 1999

ot.RandomGenerator.SetSeed(0)

formula = ['(p*g*R*S)/((ps-p)*g*D)'] 

model = ot.SymbolicFunction(['p', 'g', 'R', 'S', 'ps', 'D'], formula)
distributions = [ot.Uniform(999.8, 1000), # density of water
                 ot.Uniform(9.8, 9.81), # acceleration due to gravity
                 ot.Uniform(1, 5), # hydraulic radius
                 ot.Uniform(0.328, 0.935), # slope
                 ot.Uniform(0.328, 0.935), # density of the sediment
                 ot.Uniform(0.02, 64)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['p', 'g', 'R', 'S', 'ps', 'D']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['p', 'g', 'R', 'S', 'ps', 'D'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 999.8, b = 1000), Uniform(a = 9.8, b = 9.81), Uniform(a = 1, b = 5), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.02, b = 64), IndependentCopula(dimension = 6))
[-0.0355529,-0.0355529,-0.0326391,-0.0333942,-0.0355563,0.982746]
The least sensitive variable is:  ps
The most sensitive variable is:  D
The least sensitive variable is:  g
The most sensitive variable is:  D
First-order indices after removing least sensitive variable:  [0.0886935,0.0886935,0.128085,0.0792877,0.0886935,0.997527]
Total order indices after removing least sensitive variable:  [-1.85002e-16,1.85002e-16,0.0587889,0.153413,3.70004e-16,0.906278]
The least sensitive variable is:  ps
Modified formula:  ['(p*g*R*S)/((0.1-p)*g*D)']
Normalized first order indices using MinMaxScaler:  [[3.33912000e-06]
 [3.34041093e-06]
 [2.86474904e-03]
 [2.12329421e-03]
 [0.00000000e+00]
 [1.00000000e+00]]
Standardized first order indices using StandardScaler:  [[-0.44988567]

In [183]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Wang et al. 2015

ot.RandomGenerator.SetSeed(0)

formula = ['k0/m0'] 

model = ot.SymbolicFunction(['k0', 'm0'], formula)
distributions = [ot.Uniform(0.017, 7.13), # density of water
                 ot.Uniform(1, 10)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['k0', 'm0']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['k0', 'm0'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 0.017, b = 7.13), Uniform(a = 1, b = 10), IndependentCopula(dimension = 2))
[0.491051,0.488939]
The least sensitive variable is:  m0
The most sensitive variable is:  k0
The least sensitive variable is:  k0
The most sensitive variable is:  m0
First-order indices after removing least sensitive variable:  [1,0.00547874]
Total order indices after removing least sensitive variable:  [0.994521,3.86498e-16]
The least sensitive variable is:  m0
Modified formula:  ['k0/0.1']
Normalized first order indices using MinMaxScaler:  [[1.]
 [0.]]
Standardized first order indices using StandardScaler:  [[ 1.]
 [-1.]]


In [184]:

from openturns.usecases import ishigami_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import pandas as pd
import plotly.graph_objs as go
ot.Log.Show(ot.Log.NONE)
# Harvey et al. 2015

ot.RandomGenerator.SetSeed(0)

formula = ['g/(v*8.3e-3*((n^3)/((1-n)^2))*((d)^2))'] 

model = ot.SymbolicFunction(['g', 'v', 'n', 'd'], formula)
distributions = [ot.Uniform(9.7e-5, 4.9e-3), # density of water
                 ot.Uniform(0.328, 0.935), # slope
                 ot.Uniform(0.328, 0.935), # density of the sediment
                 ot.Uniform(1, 365)] # grain size
distribution = ot.ComposedDistribution(distributions)
print(distribution)
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)


print(sensitivityAnalysis.getFirstOrderIndices())
sensitivityIndices = sensitivityAnalysis.getFirstOrderIndices()
minIndex = np.argmin(sensitivityIndices)
maxIndex = np.argmax(sensitivityIndices)
mylab = ['g', 'v', 'n', 'd']
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

totalOrderIndices = sensitivityAnalysis.getTotalOrderIndices()
TotalIndices = sensitivityAnalysis.getTotalOrderIndices()
minIndex = np.argmin(TotalIndices)
maxIndex = np.argmax(TotalIndices)
leastSensitiveVariable = mylab[minIndex]
mostSensitiveVariable = mylab[maxIndex]
print("The least sensitive variable is: ", leastSensitiveVariable)
print("The most sensitive variable is: ", mostSensitiveVariable)

# Get the least sensitive variable from the first-order indices
leastSensitiveVariable = mylab[np.argmin(sensitivityAnalysis.getFirstOrderIndices())]

# Modify the formula to exclude the least sensitive variable
formula = [formula[0].replace(leastSensitiveVariable, '0.1')]

# Create a new symbolic function with the modified formula
model = ot.SymbolicFunction(['g', 'v', 'n', 'd'], formula)

# Perform sensitivity analysis on the modified model
sensitivityAnalysis = ot.SobolIndicesAlgorithm(distribution, 101, model)

# Display the results of the sensitivity analysis
print("First-order indices after removing least sensitive variable: ", sensitivityAnalysis.getFirstOrderIndices())
print("Total order indices after removing least sensitive variable: ", sensitivityAnalysis.getTotalOrderIndices())

print("The least sensitive variable is: ", leastSensitiveVariable)
print("Modified formula: ", formula)
# Normalize the first order indices using MinMaxScaler

minmax_scaler = MinMaxScaler()
sensitivityIndices_normalized = minmax_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Normalized first order indices using MinMaxScaler: ", sensitivityIndices_normalized)

# Normalize the first order indices using StandardScaler
standard_scaler = StandardScaler()
sensitivityIndices_standardized = standard_scaler.fit_transform(np.array(sensitivityIndices).reshape(-1, 1))
print("Standardized first order indices using StandardScaler: ", sensitivityIndices_standardized)

# Define custom colors
colors = ['rgb(57, 106, 177)', 'rgb(218, 124, 48)']

# Define the x-axis labels
x_labels = mylab

# Define the normalized and standardized values
normalized_values = sensitivityIndices_normalized.flatten()
standardized_values = sensitivityIndices_standardized.flatten()

layout = go.Layout(
    title=dict(
        font=dict(
            size=30,  # Graph title font size
            color='black'
        )
    ),
    xaxis=dict(
        title='Variables',
        tickangle=45,
        tickfont=dict(
            size=20,  # X-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.01,
        titlefont=dict(
            size=25,  # X-axis title font size
            color='black'
        ),
    ),
    yaxis=dict(
        title='Indices',
        range=[y_min - 0.7, y_max + 0.5 * (y_max - y_min)],
        tickfont=dict(
            size=20,  # Y-axis labels font size
            color='black'
        ),
        showticklabels=True,
        dtick=0.5,
        titlefont=dict(
            size=25,  # Y-axis title font size
            color='black'
        ),
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    hovermode='closest',
    plot_bgcolor='rgb(248, 248, 248)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.3,
        y=-0.4,
        font=dict(
            size=20,  # Legend font size
            color='black'
        )
    ),
)

# Modify the bar chart traces to include text font size
trace_normalized = go.Bar(
    x=x_labels,
    y=normalized_values,
    name='Normalized',
    marker=dict(
        color=colors[0]
    ),
    hovertemplate='<b>Normalized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in normalized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

trace_standardized = go.Bar(
    x=x_labels,
    y=standardized_values,
    name='Standardized',
    marker=dict(
        color=colors[1]
    ),
    hovertemplate='<b>Standardized:</b> %{y:.2f}<extra></extra>',
    text=[f'{val:.2f}' for val in standardized_values],
    textfont=dict(
        size=20,  # Text font size for values on bars
        color='black'
    ),
    textposition='auto'
)

# Create the figure and show it in Jupyter notebook
fig = go.Figure(data=[trace_normalized, trace_standardized], layout=layout)
fig.show()

ComposedDistribution(Uniform(a = 9.7e-05, b = 0.0049), Uniform(a = 0.328, b = 0.935), Uniform(a = 0.328, b = 0.935), Uniform(a = 1, b = 365), IndependentCopula(dimension = 4))
[-0.0326803,-0.036251,-0.0237317,0.0901196]
The least sensitive variable is:  v
The most sensitive variable is:  d
The least sensitive variable is:  g
The most sensitive variable is:  d
First-order indices after removing least sensitive variable:  [-0.0145873,-0.0168394,-0.0199347,0.0399207]
Total order indices after removing least sensitive variable:  [0.00797889,-6.82443e-16,0.470303,0.868935]
The least sensitive variable is:  v
Modified formula:  ['g/(0.1*8.3e-3*((n^3)/((1-n)^2))*((d)^2))']
Normalized first order indices using MinMaxScaler:  [[0.028256  ]
 [0.        ]
 [0.09906765]
 [1.        ]]
Standardized first order indices using StandardScaler:  [[-0.60925848]
 [-0.67714851]
 [-0.43912098]
 [ 1.72552797]]
